In [1]:
library(dplyr)
library(igraph)
library(foreach)
library(doSNOW)
library(Matrix)
library(EnvStats)
library(mvtnorm)
library(lfe)
library(blockTools)

data_directory <- '/nfs/sloanlab004/projects/covid_mobility_proj/data/PROCESSED_DATA/isr_market/'
n_simulations = 500
n_clusters = 26
max_n_searchers = 1000

load(paste0(
         c(data_directory, 'data_for_simulation_blocked.Rdata'),
         sep='', collapse=''
     ))




Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union



Attaching package: ‘igraph’


The following objects are masked from ‘package:dplyr’:

    as_data_frame, groups, union


The following objects are masked from ‘package:stats’:

    decompose, spectrum


The following object is masked from ‘package:base’:

    union


Loading required package: iterators

Loading required package: snow


Attaching package: ‘EnvStats’


The following object is masked from ‘package:Matrix’:

    print


The following objects are masked from ‘package:stats’:

    predict, predict.lm


The following object is masked from ‘package:base’:

    print.default




In [2]:
# Assume the probability matrix and y values just for the listings in a given assignment
ht_single_var <- function(prob_matrix, y) {
  pik <- diag(prob_matrix)
  pp <- as(outer(pik, pik, '*'), 'sparseMatrix')
  yy <- as(outer(y, y, '*'), 'sparseMatrix')
  ones_matrix <- as(array(1L, dim(pp)), 'sparseMatrix')
  diag(ones_matrix) <- 0
  
  D <- as((prob_matrix - pp) / pp, 'sparseMatrix')
  D <- as(D * ones_matrix, 'sparseMatrix')
  D <- D/prob_matrix
  D[!is.finite(D)] <- 0
  
  term_1_pre <- ((1-pik)/pik^2)*y^2
  
  term_1 <- sum(term_1_pre)
  term_2 <- sum(D*yy)
  
  return(term_1 + term_2)
}

#Assume the full set of ys, as well as an assignment vector
A2_correction <- function(prob_matrix, y, assignment) {
  sum_terms <- y^2/(2*diag(prob_matrix))
  sum_terms_masked <- assignment*sum_terms
  
  n_sum <- (prob_matrix == 0) %*% rep(1, nrow(prob_matrix))
  
  return(sum(sum_terms_masked*n_sum))
  
}

# Use the whole she-bang
covariance <- function(prob_matrix_treatment, prob_matrix_control, prob_matrix_diff, y, assignment) {
  pik_tr <- diag(prob_matrix_treatment)
  pik_ctr <- diag(prob_matrix_control)
  
  pp <- as(outer(pik_tr, pik_ctr, '*'), 'sparseMatrix')
  yy <- as(outer(y, y, '*'), 'sparseMatrix')
  ones_matrix <- as(array(1L, dim(pp)), 'sparseMatrix')
  diag(ones_matrix) <- 0
  
  D <- as((prob_matrix_diff - pp) / pp, 'sparseMatrix')
  D <- D/prob_matrix_diff
  D[!is.finite(D)] <- 0
  first_term <- sum(D*yy)

  
  y_sq_tr <- y^2/(2*diag(prob_matrix_treatment))
  y_sq_ctr <- y^2/(2*diag(prob_matrix_control))
  y_sq_mask_tr <- assignment*y_sq_tr
  y_sq_mask_ctr <- (1-assignment)*y_sq_ctr
  
  second_term = sum(y_sq_mask_tr %*% (prob_matrix_diff == 0)) + sum((prob_matrix_diff == 0) %*% y_sq_mask_ctr)
  
  return(first_term - second_term)
}

ht_var <- function(treatment_prob_mat, control_prob_mat, cross_prob_mat,
                   data, data_control, data_treatment, 
                   treatment_indices, control_indices, overall_indices,
                   outcome_var, assignment_var) {
    
    term_1 <- ht_single_var(treatment_prob_mat[treatment_indices, treatment_indices],
                            data_treatment[[outcome_var]])
    
    term_2 <- ht_single_var(control_prob_mat[control_indices, control_indices],
                            data_control[[outcome_var]])
    
    A21 <- A2_correction(treatment_prob_mat[overall_indices, overall_indices],
                         data[[outcome_var]], data[[assignment_var]])
    
    A22 <- A2_correction(control_prob_mat[overall_indices, overall_indices],
                         data[[outcome_var]], data[[assignment_var]])
    
    cov_term <- 2*covariance(treatment_prob_mat[overall_indices, overall_indices],
                             control_prob_mat[overall_indices, overall_indices],
                             cross_prob_mat[overall_indices, overall_indices],
                             data[[outcome_var]], data[[assignment_var]])
    
    var = (1/nrow(data))^2*(term_1 + term_2 + A21 + A22 - cov_term)
        
    var
}

In [3]:
hajek_estimator <- function(booked_rooms_df, unbooked_rooms_df, prob_column_name_t, prob_column_name_c,
                            threshold, outcome_column_name) {
    
    prob_column_name_sym_t <- sym(prob_column_name_t)
    prob_column_name_sym_c <- sym(prob_column_name_c)
    
    term_1_numerator <- sum((filter(booked_rooms_df, assignment == 1 & 
                                   pct_treated_neighbors >= threshold & 
                                   !!prob_column_name_sym_t != 0)[[outcome_column_name]])*(
        1/filter(booked_rooms_df, assignment == 1 & 
                 pct_treated_neighbors >= threshold & 
                 !!prob_column_name_sym_t != 0)[[prob_column_name_t]]))
    
    term_1_denominator <- sum((
        1/filter(booked_rooms_df, assignment == 1 & 
                 pct_treated_neighbors >= threshold & 
                 !!prob_column_name_sym_t != 0)[[prob_column_name_t]])) + 
    sum((
        1/filter(unbooked_rooms_df, assignment == 1 & 
                 pct_treated_neighbors >= threshold & 
                 !!prob_column_name_sym_t != 0)[[prob_column_name_t]]))
    
    
    term_2_numerator <- sum((filter(booked_rooms_df, assignment == 0 & 
                                   (1-pct_treated_neighbors) >= threshold & 
                                   !!prob_column_name_sym_c != 0)[[outcome_column_name]])*(
        1/filter(booked_rooms_df, assignment == 0 & 
                 (1-pct_treated_neighbors) >= threshold & 
                 !!prob_column_name_sym_c != 0)[[prob_column_name_c]]))
    
    term_2_denominator <- sum((
        1/filter(booked_rooms_df, assignment == 0 & 
                 (1-pct_treated_neighbors) >= threshold & 
                 !!prob_column_name_sym_c != 0)[[prob_column_name_c]])) + 
    sum((
        1/filter(unbooked_rooms_df, assignment == 0 & 
                 (1-pct_treated_neighbors) >= threshold & 
                 !!prob_column_name_sym_c != 0)[[prob_column_name_c]]))
    
    list(term_1_numerator/term_1_denominator, term_2_numerator/term_2_denominator)
}

treatment_var_bookings <- function(control_booked, control_unbooked, treatment_booked, treatment_unbooked) {
    joint_var <- var(c(rep(1, control_booked), 
                       rep(0, control_unbooked)))/(control_booked + 
                                                   control_unbooked) + 
    var(c(rep(1, treatment_booked), 
          rep(0, treatment_unbooked)))/(treatment_booked + 
                                        treatment_unbooked)
    
    joint_var
}

In [4]:
set.seed(15840)
sim_info <- vector("list", n_simulations)
searcher_preferences_uniform <- vector("list", n_simulations)
lat_search_bounds <- vector("list", n_simulations)
lon_search_bounds <- vector("list", n_simulations)
max_lat = quantile(airbnb_listings$latitude, .9975)
min_lat = quantile(airbnb_listings$latitude, .0025)
max_lon = quantile(airbnb_listings$longitude, .9975)
min_lon = quantile(airbnb_listings$longitude, .0025)
min_acc = min(airbnb_listings$accommodates)
max_acc = 4

cl <- makeSOCKcluster(n_clusters)
registerDoSNOW(cl)
sim_info <- foreach(i=1:n_simulations, .packages=c('EnvStats', 'Matrix', 'dplyr', 'mvtnorm'),
                    .multicombine= TRUE, .maxcombine = 2) %dopar% {
    sim_info <- list(search_weights = invisible(prop.table(runif(9))),
                     lat_search_bounds = invisible(matrix(runif(2*max_n_searchers, min=min_lat, 
                                                                max=max_lat),
                                                          ncol = 2)),
                     lon_search_bounds = invisible(matrix(runif(2*max_n_searchers, min=min_lon, 
                                                                max=max_lon),
                                                          ncol = 2)),
                     min_search_acc = runif(max_n_searchers, min=min_acc, max=max_acc),
                     search_quality_cutoff = runif(max_n_searchers, min=-2, max=2),
                     unif_searcher_weights = apply(matrix(runif(9*max_n_searchers), nrow=max_n_searchers), 
                                                   MARGIN=1, FUN=prop.table),
                     income = rnorm(max_n_searchers),
                     user_preferences_normal = rmvnorm(mean=rep(0,9), n=max_n_searchers)
                     )
}
stopCluster(cl)

extreme_value_distribution = as(matrix(revd(n=nrow(airbnb_listings)*max_n_searchers),
                                                            nrow=max_n_searchers), 'sparseMatrix')

In [5]:
set.seed(15840)
airbnb_listings$unobserved_component <- rnorm(nrow(airbnb_listings))
airbnb_listings$price_scaled = as.numeric(scale(airbnb_listings$price, scale=TRUE))
airbnb_listings$rn <- seq(1, nrow(airbnb_listings), 1)

In [6]:
simulate_ind_randomization <- function(data, pct, effect_type, effect_size, sim_info,
                                      n_searchers, sim_info_index, consumer_type, ev) {
    data$assignment <- rbinom(nrow(data), 1, pct)
    
    if(effect_type == 'price') {
        data <- data %>% 
        mutate(price_scaled = as.numeric(scale(price, scale=TRUE))) %>%
        mutate(price_scaled = ifelse(assignment == 1, price_scaled-effect_size, price_scaled)) 
    } else{
        data <- data %>% 
        mutate(unobserved_component = ifelse(assignment == 1, 
                                             unobserved_component+effect_size, 
                                             unobserved_component))
    }
    
    data %>% 
    mutate(search_quality = sim_info[[sim_info_index]]$search_weights[1]*reviews_scaled + 
           sim_info[[sim_info_index]]$search_weights[2]*satisfaction_scaled + 
           sim_info[[sim_info_index]]$search_weights[3]*bed_scaled + 
           sim_info[[sim_info_index]]$search_weights[4]*bath_scaled + 
           sim_info[[sim_info_index]]$search_weights[5]*minstay_scaled -
           sim_info[[sim_info_index]]$search_weights[6]*price_scaled + 
           sim_info[[sim_info_index]]$search_weights[7]*`Entire home/apt` + 
           sim_info[[sim_info_index]]$search_weights[8]*`Private room` + 
           sim_info[[sim_info_index]]$search_weights[9]*`Shared room`) -> data
    
    min_search = sim_info[[sim_info_index]]$min_search_acc
  
    #Initialize an empty df of the booked rooms
    booked_rooms <- filter(data, room_id == -1)
  
    for (j in 1:n_searchers) {
        min_search_lon <- min(sim_info[[sim_info_index]]$lon_search_bounds[j,])
        max_search_lon <- max(sim_info[[sim_info_index]]$lon_search_bounds[j,])
        min_search_lat <- min(sim_info[[sim_info_index]]$lat_search_bounds[j,])
        max_search_lat <- max(sim_info[[sim_info_index]]$lat_search_bounds[j,])
        min_search_acc <- floor(min_search[j])
        
        if(consumer_type == 'uniform') {
            searcher_weights <- sim_info[[sim_info_index]]$unif_searcher_weights[,j]
        } else {
            searcher_weights <- sim_info[[sim_info_index]]$user_preferences_normal[j,]
        }
        
        if(consumer_type != 'uniform') {
            data$ev_error <- extreme_value_distribution[j,][data$rn]
            income <- sim_info[[sim_info_index]]$income[j]
            
            data %>% 
            filter(latitude >= min_search_lat & latitude <= max_search_lat &
                   longitude >= min_search_lon & longitude <= max_search_lon & 
                   accommodates >= min_search_acc) %>%
            arrange(desc(search_quality)) %>% 
            head(n=10) %>%
            mutate(searcher_quality = searcher_weights[1]*reviews_scaled + 
                   searcher_weights[2]*satisfaction_scaled + 
                   searcher_weights[3]*bed_scaled + 
                   searcher_weights[4]*bath_scaled - 
                   searcher_weights[5]*minstay_scaled +
                   abs(searcher_weights[6])*(income-price_scaled) + 
                   searcher_weights[7]*`Entire home/apt` +
                   searcher_weights[8]*`Private room` + 
                   searcher_weights[9]*`Shared room` + 
                   unobserved_component + 
                   ev_error) %>% 
            arrange(desc(searcher_quality)) %>% 
            head(n=1) -> searcher_selection
            
            if(nrow(searcher_selection) == 1 & searcher_selection$searcher_quality[1] >= 0) {
                booked_rooms <- rbind(booked_rooms, searcher_selection)
                data <- filter(data, room_id != searcher_selection$room_id[1])
                }
            
            } else {
            
            search_quality_cutoff <- sim_info[[sim_info_index]]$search_quality_cutoff[j]
            data %>% 
            filter(latitude >= min_search_lat & latitude <= max_search_lat &
                   longitude >= min_search_lon & longitude <= max_search_lon & 
                   accommodates >= min_search_acc) %>%
            arrange(desc(search_quality)) %>% 
            head(n=10) %>%
            mutate(searcher_quality = searcher_weights[1]*reviews_scaled + 
                   searcher_weights[2]*satisfaction_scaled + 
                   searcher_weights[3]*bed_scaled + 
                   searcher_weights[4]*bath_scaled - 
                   searcher_weights[5]*minstay_scaled - 
                   abs(searcher_weights[6])*price_scaled + 
                   searcher_weights[7]*`Entire home/apt` +
                   searcher_weights[8]*`Private room` + 
                   searcher_weights[9]*`Shared room`) %>% 
            arrange(desc(searcher_quality)) %>% 
            head(n=1) -> searcher_selection
            
            if(nrow(searcher_selection) == 1 & searcher_selection$searcher_quality[1] >= search_quality_cutoff) {
                booked_rooms <- rbind(booked_rooms, searcher_selection)
                data <- filter(data, room_id != searcher_selection$room_id[1])
            }
        }
    }
  
    treatment_booked <- nrow(filter(booked_rooms, assignment == 1))
    treatment_unbooked <- nrow(filter(data, assignment == 1))
    treatment_revenue <- sum(filter(booked_rooms, assignment == 1)$price)
    control_booked <- nrow(filter(booked_rooms, assignment == 0))
    control_unbooked <- nrow(filter(data, assignment == 0))
    control_revenue <- sum(filter(booked_rooms, assignment == 0)$price)
  
    p_value <- ifelse(pct %in% c(0,1), 'NA', prop.test(c(treatment_booked, control_booked), 
                                                       c(treatment_booked + treatment_unbooked, 
                                                       control_booked + control_unbooked))$p.value)
  
    treatment_prices <- c(filter(booked_rooms, assignment == 1)$price, rep(0, treatment_unbooked))
    control_prices <- c(filter(booked_rooms, assignment == 0)$price, rep(0, control_unbooked))
  
    price_p_value <- ifelse(pct %in% c(0,1), 'NA', wilcox.test(treatment_prices, control_prices)$p.value)
    
    data$revenue <- 0
    booked_rooms$revenue <- booked_rooms$price
    data$booked <- 0
    booked_rooms$booked <- 1
    
    reg_data <- rbind(data[1:82], booked_rooms[1:82])
    
    booking_diff_var <- treatment_var_bookings(control_booked, control_unbooked,
                                               treatment_booked, treatment_unbooked)
    
    price_diff_var <- var(treatment_prices)/length(treatment_prices) + var(control_prices)/length(control_prices)
    
  
    data.frame(treatment_booked = treatment_booked, 
               treatment_unbooked = treatment_unbooked, 
               treatment_booking_rate = treatment_booked/(treatment_unbooked + treatment_booked),
               treatment_revenue = treatment_revenue,
               treatment_revenue_per_user = treatment_revenue/(treatment_unbooked + treatment_booked),
               control_booked = control_booked,
               control_unbooked = control_unbooked,
               control_booking_rate = control_booked/(control_unbooked + control_booked),
               control_revenue = control_revenue,
               control_revenue_per_user = control_revenue/(control_unbooked + control_booked),
               p_value = p_value,
               price_p_value = price_p_value,
               booking_diff_var = booking_diff_var,
               price_diff_var = price_diff_var
  )
}

In [56]:
simulate_ind_randomization_wash <- function(data, pct, effect_type, effect_size, sim_info,
                                      n_searchers, sim_info_index, consumer_type, ev) {
    data$assignment <- rbinom(nrow(data), 1, pct)
    
    if(effect_type == 'price') {
        data <- data %>% 
        mutate(price_scaled = as.numeric(scale(price, scale=TRUE))) %>%
        mutate(price_scaled = ifelse(assignment == 1, price_scaled-effect_size, price_scaled)) 
    } else{
        data <- data %>% 
        mutate(unobserved_component = ifelse(assignment == 1, 
                                             unobserved_component+effect_size, 
                                             unobserved_component))
    }
    
    data %>% 
    mutate(search_quality = sim_info[[sim_info_index]]$search_weights[1]*reviews_scaled + 
           sim_info[[sim_info_index]]$search_weights[2]*satisfaction_scaled + 
           sim_info[[sim_info_index]]$search_weights[3]*bed_scaled + 
           sim_info[[sim_info_index]]$search_weights[5]*minstay_scaled -
           sim_info[[sim_info_index]]$search_weights[6]*price_scaled + 
           sim_info[[sim_info_index]]$search_weights[7]*`Entire home/apt` + 
           sim_info[[sim_info_index]]$search_weights[8]*`Private room` + 
           sim_info[[sim_info_index]]$search_weights[9]*`Shared room`) -> data
    
    min_search = sim_info[[sim_info_index]]$min_search_acc
  
    #Initialize an empty df of the booked rooms
    booked_rooms <- filter(data, room_id == -1)
  
    for (j in 1:n_searchers) {
        min_search_lon <- min(sim_info[[sim_info_index]]$lon_search_bounds[j,])
        max_search_lon <- max(sim_info[[sim_info_index]]$lon_search_bounds[j,])
        min_search_lat <- min(sim_info[[sim_info_index]]$lat_search_bounds[j,])
        max_search_lat <- max(sim_info[[sim_info_index]]$lat_search_bounds[j,])
        min_search_acc <- floor(min_search[j])
        
        if(consumer_type == 'uniform') {
            searcher_weights <- sim_info[[sim_info_index]]$unif_searcher_weights[,j]
        } else {
            searcher_weights <- sim_info[[sim_info_index]]$user_preferences_normal[j,]
        }
        
        if(consumer_type != 'uniform') {
            data$ev_error <- extreme_value_distribution[j,][data$rn]
            income <- sim_info[[sim_info_index]]$income[j]
            
            data %>% 
            filter(latitude >= min_search_lat & latitude <= max_search_lat &
                   longitude >= min_search_lon & longitude <= max_search_lon & 
                   accommodates >= min_search_acc) %>%
            arrange(desc(search_quality)) %>% 
            head(n=10) %>%
            mutate(searcher_quality = searcher_weights[1]*reviews_scaled + 
                   searcher_weights[2]*satisfaction_scaled + 
                   searcher_weights[3]*bed_scaled - 
                   searcher_weights[5]*minstay_scaled +
                   abs(searcher_weights[6])*(income-price_scaled) + 
                   searcher_weights[7]*`Entire home/apt` +
                   searcher_weights[8]*`Private room` + 
                   searcher_weights[9]*`Shared room` + 
                   unobserved_component + 
                   ev_error) %>% 
            arrange(desc(searcher_quality)) %>% 
            head(n=1) -> searcher_selection
            
            if(nrow(searcher_selection) == 1 & searcher_selection$searcher_quality[1] >= 0) {
                booked_rooms <- rbind(booked_rooms, searcher_selection)
                data <- filter(data, room_id != searcher_selection$room_id[1])
                }
            
            } else {
            
            search_quality_cutoff <- sim_info[[sim_info_index]]$search_quality_cutoff[j]
            data %>% 
            filter(latitude >= min_search_lat & latitude <= max_search_lat &
                   longitude >= min_search_lon & longitude <= max_search_lon & 
                   accommodates >= min_search_acc) %>%
            arrange(desc(search_quality)) %>% 
            head(n=10) %>%
            mutate(searcher_quality = searcher_weights[1]*reviews_scaled + 
                   searcher_weights[2]*satisfaction_scaled + 
                   searcher_weights[3]*bed_scaled - 
                   searcher_weights[5]*minstay_scaled - 
                   abs(searcher_weights[6])*price_scaled + 
                   searcher_weights[7]*`Entire home/apt` +
                   searcher_weights[8]*`Private room` + 
                   searcher_weights[9]*`Shared room`) %>% 
            arrange(desc(searcher_quality)) %>% 
            head(n=1) -> searcher_selection
            
            if(nrow(searcher_selection) == 1 & searcher_selection$searcher_quality[1] >= search_quality_cutoff) {
                booked_rooms <- rbind(booked_rooms, searcher_selection)
                data <- filter(data, room_id != searcher_selection$room_id[1])
            }
        }
    }
  
    treatment_booked <- nrow(filter(booked_rooms, assignment == 1))
    treatment_unbooked <- nrow(filter(data, assignment == 1))
    treatment_revenue <- sum(filter(booked_rooms, assignment == 1)$price)
    control_booked <- nrow(filter(booked_rooms, assignment == 0))
    control_unbooked <- nrow(filter(data, assignment == 0))
    control_revenue <- sum(filter(booked_rooms, assignment == 0)$price)
  
    p_value <- ifelse(pct %in% c(0,1), 'NA', prop.test(c(treatment_booked, control_booked), 
                                                       c(treatment_booked + treatment_unbooked, 
                                                       control_booked + control_unbooked))$p.value)
  
    treatment_prices <- c(filter(booked_rooms, assignment == 1)$price, rep(0, treatment_unbooked))
    control_prices <- c(filter(booked_rooms, assignment == 0)$price, rep(0, control_unbooked))
  
    price_p_value <- ifelse(pct %in% c(0,1), 'NA', wilcox.test(treatment_prices, control_prices)$p.value)
    
    data$revenue <- 0
    booked_rooms$revenue <- booked_rooms$price
    data$booked <- 0
    booked_rooms$booked <- 1
    
    reg_data <- rbind(data[names(data)], booked_rooms[names(data)])
    
    booking_diff_var <- treatment_var_bookings(control_booked, control_unbooked,
                                               treatment_booked, treatment_unbooked)
    
    price_diff_var <- var(treatment_prices)/length(treatment_prices) + var(control_prices)/length(control_prices)
    
  
    data.frame(treatment_booked = treatment_booked, 
               treatment_unbooked = treatment_unbooked, 
               treatment_booking_rate = treatment_booked/(treatment_unbooked + treatment_booked),
               treatment_revenue = treatment_revenue,
               treatment_revenue_per_user = treatment_revenue/(treatment_unbooked + treatment_booked),
               control_booked = control_booked,
               control_unbooked = control_unbooked,
               control_booking_rate = control_booked/(control_unbooked + control_booked),
               control_revenue = control_revenue,
               control_revenue_per_user = control_revenue/(control_unbooked + control_booked),
               p_value = p_value,
               price_p_value = price_p_value,
               booking_diff_var = booking_diff_var,
               price_diff_var = price_diff_var
  )
}

In [8]:
simulate_graph_randomization <- function(data, pct, effect_type, effect_size, sim_info,
                                      n_searchers, sim_info_index, consumer_type, ev,
                                        community, graph, block,
                                        prob_vars,
                                        hajek_prob_matrices) {
    
    sink(paste0(
         c(data_directory, i,'.txt'),
         sep='', collapse=''
     ))

    n_communities <- length(unique(community))
    assignments <- rep(0, n_communities)
    assignments[as.numeric(
        as.character(unlist(blockTools::assignment(block)[[1]][[1]]['Treatment 1'])))] <- 1
    vertsInAssignments <- community %in% which(assignments == 1)
  
    data$cluster_membership <- community
    data <- data %>% mutate(assignment = ifelse(assignments[cluster_membership] == 1,
                                       1, 0))
    
    if(effect_type == 'price') {
        data <- data %>% 
        mutate(price_scaled = as.numeric(scale(price, scale=TRUE))) %>%
        mutate(price_scaled = ifelse(assignment == 1, price_scaled-effect_size, price_scaled)) 
    } else{
        data <- data %>% 
        mutate(unobserved_component = ifelse(assignment == 1, 
                                             unobserved_component+effect_size, 
                                             unobserved_component))
    }
    
    data %>% 
    mutate(search_quality = sim_info[[sim_info_index]]$search_weights[1]*reviews_scaled + 
           sim_info[[sim_info_index]]$search_weights[2]*satisfaction_scaled + 
           sim_info[[sim_info_index]]$search_weights[3]*bed_scaled + 
           sim_info[[sim_info_index]]$search_weights[4]*bath_scaled + 
           sim_info[[sim_info_index]]$search_weights[5]*minstay_scaled -
           sim_info[[sim_info_index]]$search_weights[6]*price_scaled + 
           sim_info[[sim_info_index]]$search_weights[7]*`Entire home/apt` + 
           sim_info[[sim_info_index]]$search_weights[8]*`Private room` + 
           sim_info[[sim_info_index]]$search_weights[9]*`Shared room`) -> data
    
    data$pct_treated_neighbors <- sapply(1:vcount(graph), FUN=function(i){
        neigh <- neighbors(graph, i)
        sum(vertsInAssignments[neigh])/length(neigh)
    })
    
    min_search = sim_info[[sim_info_index]]$min_search_acc
  
    booked_rooms <- filter(data, room_id == -1)
  
  
    for (j in 1:n_searchers) {
        
        min_search_lon <- min(sim_info[[sim_info_index]]$lon_search_bounds[j,])
        max_search_lon <- max(sim_info[[sim_info_index]]$lon_search_bounds[j,])
        min_search_lat <- min(sim_info[[sim_info_index]]$lat_search_bounds[j,])
        max_search_lat <- max(sim_info[[sim_info_index]]$lat_search_bounds[j,])
        min_search_acc <- floor(min_search[j])
        
        if(consumer_type == 'uniform') {
            searcher_weights <- sim_info[[sim_info_index]]$unif_searcher_weights[,j]
        } else {
            searcher_weights <- sim_info[[sim_info_index]]$user_preferences_normal[j,]
        }
        
        if(consumer_type != 'uniform') {
            data$ev_error <- extreme_value_distribution[j,][data$rn]
            income <- sim_info[[sim_info_index]]$income[j]
            
            data %>% 
            filter(latitude >= min_search_lat & latitude <= max_search_lat &
                   longitude >= min_search_lon & longitude <= max_search_lon & 
                   accommodates >= min_search_acc) %>%
            arrange(desc(search_quality)) %>% 
            head(n=10) %>%
            mutate(searcher_quality = searcher_weights[1]*reviews_scaled + 
                   searcher_weights[2]*satisfaction_scaled + 
                   searcher_weights[3]*bed_scaled + 
                   searcher_weights[4]*bath_scaled - 
                   searcher_weights[5]*minstay_scaled +
                   abs(searcher_weights[6])*(income-price_scaled) + 
                   searcher_weights[7]*`Entire home/apt` +
                   searcher_weights[8]*`Private room` + 
                   searcher_weights[9]*`Shared room` + 
                   unobserved_component + 
                   ev_error) %>% 
            arrange(desc(searcher_quality)) %>% 
            head(n=1) -> searcher_selection
            
            if(nrow(searcher_selection) == 1 & searcher_selection$searcher_quality[1] >= 0) {
                booked_rooms <- rbind(booked_rooms, searcher_selection)
                data <- filter(data, room_id != searcher_selection$room_id[1])
                }
            
            } else {
            
            search_quality_cutoff <- sim_info[[sim_info_index]]$search_quality_cutoff[j]
            data %>% 
            filter(latitude >= min_search_lat & latitude <= max_search_lat &
                   longitude >= min_search_lon & longitude <= max_search_lon & 
                   accommodates >= min_search_acc) %>%
            arrange(desc(search_quality)) %>% 
            head(n=10) %>%
            mutate(searcher_quality = searcher_weights[1]*reviews_scaled + 
                   searcher_weights[2]*satisfaction_scaled + 
                   searcher_weights[3]*bed_scaled + 
                   searcher_weights[4]*bath_scaled - 
                   searcher_weights[5]*minstay_scaled - 
                   abs(searcher_weights[6])*price_scaled + 
                   searcher_weights[7]*`Entire home/apt` +
                   searcher_weights[8]*`Private room` + 
                   searcher_weights[9]*`Shared room`) %>% 
            arrange(desc(searcher_quality)) %>% 
            head(n=1) -> searcher_selection
            
            if(nrow(searcher_selection) == 1 & searcher_selection$searcher_quality[1] >= search_quality_cutoff) {
                booked_rooms <- rbind(booked_rooms, searcher_selection)
                data <- filter(data, room_id != searcher_selection$room_id[1])
            }
        }
    }
    
    data <- data %>% 
      mutate(eff_treat_50 = ifelse(assignment == 1 & pct_treated_neighbors >= .5, 1, 0),
             eff_treat_75 = ifelse(assignment == 1 & pct_treated_neighbors >= .75, 1, 0),
             eff_treat_95 = ifelse(assignment == 1 & pct_treated_neighbors >= .95, 1, 0),
             eff_control_50 = ifelse(assignment == 0 & (1-pct_treated_neighbors) >= .5, 1, 0),
             eff_control_75 = ifelse(assignment == 0 & (1-pct_treated_neighbors) >= .75, 1, 0),
             eff_control_95 = ifelse(assignment == 0 & (1-pct_treated_neighbors) >= .95, 1, 0),
             revenue = 0,
             booked = 0)
    
    booked_rooms <- booked_rooms %>% 
      mutate(eff_treat_50 = ifelse(assignment == 1 & pct_treated_neighbors >= .5, 1, 0),
             eff_treat_75 = ifelse(assignment == 1 & pct_treated_neighbors >= .75, 1, 0),
             eff_treat_95 = ifelse(assignment == 1 & pct_treated_neighbors >= .95, 1, 0),
             eff_control_50 = ifelse(assignment == 0 & (1-pct_treated_neighbors) >= .5, 1, 0),
             eff_control_75 = ifelse(assignment == 0 & (1-pct_treated_neighbors) >= .75, 1, 0),
             eff_control_95 = ifelse(assignment == 0 & (1-pct_treated_neighbors) >= .95, 1, 0),
             revenue = price,
             booked = 1)
    
  
    treatment_booked <- nrow(filter(booked_rooms, assignment == 1))
    treatment_unbooked <- nrow(filter(data, assignment == 1))
    treatment_revenue <- sum(filter(booked_rooms, assignment == 1)$price)
    control_booked <- nrow(filter(booked_rooms, assignment == 0))
    control_unbooked <- nrow(filter(data, assignment == 0))
    control_revenue <- sum(filter(booked_rooms, assignment == 0)$price)
    
    eff_treatment_booked_50 <- nrow(filter(booked_rooms, eff_treat_50 == 1))
    eff_treatment_unbooked_50 <- nrow(filter(data, eff_treat_50 == 1))
    eff_treatment_revenue_50 <- sum(filter(booked_rooms, eff_treat_50 == 1)$price)
    eff_control_booked_50 <- nrow(filter(booked_rooms, eff_control_50 == 1))
    eff_control_unbooked_50 <- nrow(filter(data, eff_control_50 == 1))
    eff_control_revenue_50 <- sum(filter(booked_rooms, eff_control_50 == 1)$price)
  
    eff_treatment_booked_75 <- nrow(filter(booked_rooms, eff_treat_75 == 1))
    eff_treatment_unbooked_75 <- nrow(filter(data, eff_treat_75 == 1))
    eff_treatment_revenue_75 <- sum(filter(booked_rooms, eff_treat_75 == 1)$price)
    eff_control_booked_75 <- nrow(filter(booked_rooms, eff_control_75 == 1))
    eff_control_unbooked_75 <- nrow(filter(data, eff_control_75 == 1))
    eff_control_revenue_75 <- sum(filter(booked_rooms, eff_control_75 == 1)$price)
  
    eff_treatment_booked_95 <- nrow(filter(booked_rooms, eff_treat_95 == 1))
    eff_treatment_unbooked_95 <- nrow(filter(data, eff_treat_95 == 1))
    eff_treatment_revenue_95 <- sum(filter(booked_rooms, eff_treat_95 == 1)$price)
    eff_control_booked_95 <- nrow(filter(booked_rooms, eff_control_95 == 1))
    eff_control_unbooked_95 <- nrow(filter(data, eff_control_95 == 1))
    eff_control_revenue_95 <- sum(filter(booked_rooms, eff_control_95 == 1)$price)
    
    treatment_prices <- c(filter(booked_rooms, assignment == 1)$price, rep(0, treatment_unbooked))
    eff_treatment_prices_50 <- c(filter(booked_rooms, eff_treat_50 == 1)$price, rep(0, eff_treatment_unbooked_50))
    eff_treatment_prices_75 <- c(filter(booked_rooms, eff_treat_75 == 1)$price, rep(0, eff_treatment_unbooked_75))
    eff_treatment_prices_95 <- c(filter(booked_rooms, eff_treat_95 == 1)$price, rep(0, eff_treatment_unbooked_95))
    
    control_prices <- c(filter(booked_rooms, assignment == 0)$price, rep(0, control_unbooked))
    eff_control_prices_50 <- c(filter(booked_rooms, eff_control_50 == 1)$price, rep(0, eff_control_unbooked_50))
    eff_control_prices_75 <- c(filter(booked_rooms, eff_control_75 == 1)$price, rep(0, eff_control_unbooked_75))
    eff_control_prices_95 <- c(filter(booked_rooms, eff_control_95 == 1)$price, rep(0, eff_control_unbooked_95)) 
    
    reg_data <- rbind(data %>% 
                      dplyr::select(rn, booked, revenue, assignment, cluster_membership,
                                    eff_control_50, eff_treat_50, eff_control_75, eff_treat_75,
                                    eff_control_95, eff_treat_95),
                      booked_rooms %>%
                      dplyr::select(rn, booked, revenue, assignment, cluster_membership,
                                    eff_control_50, eff_treat_50, eff_control_75, eff_treat_75,
                                    eff_control_95, eff_treat_95)) %>% arrange(rn)
    
    booking_diff_var <- treatment_var_bookings(control_booked, control_unbooked,
                                               treatment_booked, treatment_unbooked)
    price_diff_var <- var(treatment_prices)/length(treatment_prices) + var(control_prices)/length(control_prices)
    
    booking_diff_50_var <- treatment_var_bookings(eff_control_booked_50, eff_control_unbooked_50,
                                               eff_treatment_booked_50, eff_treatment_unbooked_50)
    price_diff_50_var <- var(eff_treatment_prices_50)/length(eff_treatment_prices_50) + 
                         var(eff_control_prices_50)/length(eff_control_prices_50)
    
    booking_diff_75_var <- treatment_var_bookings(eff_control_booked_75, eff_control_unbooked_75,
                                               eff_treatment_booked_75, eff_treatment_unbooked_75)
    price_diff_75_var <- var(eff_treatment_prices_75)/length(eff_treatment_prices_75) + 
                         var(eff_control_prices_75)/length(eff_control_prices_75)

    booking_diff_95_var <- treatment_var_bookings(eff_control_booked_95, eff_control_unbooked_95,
                                               eff_treatment_booked_95, eff_treatment_unbooked_95)
    price_diff_95_var <- var(eff_treatment_prices_95)/length(eff_treatment_prices_95) + 
                         var(eff_control_prices_95)/length(eff_control_prices_95)
    
    p_value <- ifelse(pct %in% c(0,1) | (treatment_booked + treatment_unbooked) <= 0 | 
                      (control_booked + control_unbooked) <= 0, NA, 
                      prop.test(c(treatment_booked, control_booked), 
                                c(treatment_booked + treatment_unbooked, 
                                  control_booked + control_unbooked))$p.value)
    
    if(pct %in% c(0,1) | (eff_treatment_booked_75 + eff_treatment_unbooked_75) <= 0 | 
                             (eff_control_booked_75 + eff_control_unbooked_75) <= 0) {
        eff_p_value_75 <- 'NA'
    } else {
        eff_p_value_75 <- prop.test(c(eff_treatment_booked_75, eff_control_booked_75), 
                                           c(eff_treatment_booked_75 + eff_treatment_unbooked_75, 
                                             eff_control_booked_75 + eff_control_unbooked_75))$p.value
    }
    
    if(pct %in% c(0,1) | (eff_treatment_booked_50 + eff_treatment_unbooked_50) <= 0 | 
                             (eff_control_booked_50 + eff_control_unbooked_50) <= 0) {
        eff_p_value_50 <- 'NA'
    } else {
        eff_p_value_50 <- prop.test(c(eff_treatment_booked_50, eff_control_booked_50), 
                                       c(eff_treatment_booked_50 + eff_treatment_unbooked_50, 
                                         eff_control_booked_50 + eff_control_unbooked_50))$p.value
    }
    
    if(pct %in% c(0,1) | (eff_treatment_booked_95 + eff_treatment_unbooked_95) <= 0 |
                             (eff_control_booked_95 + eff_control_unbooked_95) <= 0) {
        eff_p_value_95 <- 'NA'
    } else{
        eff_p_value_95 <- prop.test(c(eff_treatment_booked_95, eff_control_booked_95), 
                                       c(eff_treatment_booked_95 + eff_treatment_unbooked_95, 
                                         eff_control_booked_95 + eff_control_unbooked_95))$p.value
    }
    
    price_p_value <- ifelse(pct %in% c(0,1), 'NA', wilcox.test(treatment_prices, control_prices)$p.value)
    
    if(pct %in% c(0,1) | length(eff_treatment_prices_50) <= 1 | length(eff_control_prices_50) <= 1) {
        eff_price_p_value_50 <- 'NA'
    } else {
        eff_price_p_value_50 <- wilcox.test(eff_treatment_prices_50, eff_control_prices_50)$p.value
    }
    
    if(pct %in% c(0,1) | length(eff_treatment_prices_75) <= 1 | length(eff_control_prices_75) <= 1) {
        eff_price_p_value_75 <- 'NA'
    } else {
        eff_price_p_value_75 <- wilcox.test(eff_treatment_prices_75, eff_control_prices_75)$p.value
    }
    
    if(pct %in% c(0,1) | length(eff_treatment_prices_95) <= 1 | length(eff_control_prices_95) <= 1) {
        eff_price_p_value_95 <- 'NA'
    } else {
        eff_price_p_value_95 <- wilcox.test(eff_treatment_prices_95, eff_control_prices_95)$p.value
    }
    
    if(length(eff_treatment_prices_50) <= 1 | length(eff_control_prices_50) <= 1) {
        hajek_estimator_50 <- 'NA'
        price_hajek_estimator_50 <- 'NA'
        } else {
        hajek_estimator_50 <- hajek_estimator(booked_rooms, data, prob_vars[1], prob_vars[2], .5, 'booked')
        price_hajek_estimator_50 <- hajek_estimator(booked_rooms, data, prob_vars[1], prob_vars[2], .5, 'revenue')
    }

    if(length(eff_treatment_prices_75) <= 1 | length(eff_control_prices_75) <= 1) {
        hajek_estimator_75 <- 'NA'
        price_hajek_estimator_75 <- 'NA'
        } else {
        hajek_estimator_75 <- hajek_estimator(booked_rooms, data, prob_vars[3], prob_vars[4], .75, 'booked')
        price_hajek_estimator_75 <- hajek_estimator(booked_rooms, data, prob_vars[3], prob_vars[4], .75, 'revenue')
    }    

    if(length(eff_treatment_prices_95) <= 1 | length(eff_control_prices_95) <= 1) {
        hajek_estimator_95 <- 'NA'
        price_hajek_estimator_95 <- 'NA'
        } else {
        hajek_estimator_95 <- hajek_estimator(booked_rooms, data, prob_vars[5], prob_vars[6], .95, 'booked')
        price_hajek_estimator_95 <- hajek_estimator(booked_rooms, data, prob_vars[5], prob_vars[6], .95, 'revenue') 
    }    
    
    if(length(eff_treatment_prices_50) <= 1 | length(eff_control_prices_50) <= 1) {
        hajek_var_50_bookings <- 'NA'
        hajek_var_50_price <- 'NA'
        } else {
        reg_data_eff_50_control <- filter(reg_data, eff_control_50 == 1) %>% 
        mutate(booked_resid = booked - hajek_estimator_50[[2]],
               revenue_resid = revenue - price_hajek_estimator_50[[2]])
        reg_data_eff_50_treatment <- filter(reg_data, eff_treat_50 == 1) %>% 
        mutate(booked_resid = booked - hajek_estimator_50[[1]],
               revenue_resid = revenue - price_hajek_estimator_50[[1]])
        reg_data_eff_50 <- rbind(reg_data_eff_50_control, reg_data_eff_50_treatment)
        
        indices_50 <- c(reg_data_eff_50_treatment$rn, reg_data_eff_50_control$rn)
        
        hajek_var_50_bookings <- ht_var(hajek_prob_matrices[[1]][[1]],
                                        hajek_prob_matrices[[1]][[2]],
                                        hajek_prob_matrices[[1]][[3]],
                                        reg_data_eff_50, reg_data_eff_50_control, reg_data_eff_50_treatment,
                                        reg_data_eff_50_treatment$rn, reg_data_eff_50_control$rn, indices_50,
                                        'booked_resid', 'eff_treat_50')
        
        hajek_var_50_price <- ht_var(hajek_prob_matrices[[1]][[1]],
                                     hajek_prob_matrices[[1]][[2]],
                                     hajek_prob_matrices[[1]][[3]],
                                     reg_data_eff_50, reg_data_eff_50_control, reg_data_eff_50_treatment,
                                     reg_data_eff_50_treatment$rn, reg_data_eff_50_control$rn, indices_50,
                                     'revenue_resid', 'eff_treat_50')
    }
    
    if(length(eff_treatment_prices_75) <= 1 | length(eff_control_prices_75) <= 1) {
        hajek_var_75_bookings <- 'NA'
        hajek_var_75_price <- 'NA'
        } else {
        
        reg_data_eff_75_control <- filter(reg_data, eff_control_75 == 1) %>% 
        mutate(booked_resid = booked - hajek_estimator_75[[2]],
               revenue_resid = revenue - price_hajek_estimator_75[[2]]) 
        reg_data_eff_75_treatment <- filter(reg_data, eff_treat_75 == 1) %>% 
        mutate(booked_resid = booked - hajek_estimator_75[[1]],
               revenue_resid = revenue - price_hajek_estimator_75[[1]]) 
        reg_data_eff_75 <- rbind(reg_data_eff_75_control, reg_data_eff_75_treatment)
        
        indices_75 <- c(reg_data_eff_75_treatment$rn, reg_data_eff_75_control$rn)
        
        hajek_var_75_bookings <- ht_var(hajek_prob_matrices[[2]][[1]],
                                        hajek_prob_matrices[[2]][[2]],
                                        hajek_prob_matrices[[2]][[3]],
                                        reg_data_eff_75, reg_data_eff_75_control, reg_data_eff_75_treatment,
                                        reg_data_eff_75_treatment$rn, reg_data_eff_75_control$rn, indices_75,
                                        'booked_resid', 'eff_treat_75')
        
        hajek_var_75_price <- ht_var(hajek_prob_matrices[[2]][[1]],
                                     hajek_prob_matrices[[2]][[2]],
                                     hajek_prob_matrices[[2]][[3]],
                                     reg_data_eff_75, reg_data_eff_75_control, reg_data_eff_75_treatment,
                                     reg_data_eff_75_treatment$rn, reg_data_eff_75_control$rn, indices_75,
                                     'revenue_resid', 'eff_treat_75')
    }
    
    if(length(eff_treatment_prices_95) <= 1 | length(eff_control_prices_95) <= 1) {
        hajek_var_95_bookings <- 'NA'
        hajek_var_95_price <- 'NA'
        } else {
        
        reg_data_eff_95_control <- filter(reg_data, eff_control_95 == 1) %>% 
        mutate(booked_resid = booked - hajek_estimator_95[[2]],
               revenue_resid = revenue - price_hajek_estimator_95[[2]]) 
        reg_data_eff_95_treatment <- filter(reg_data, eff_treat_95 == 1) %>% 
        mutate(booked_resid = booked - hajek_estimator_95[[1]],
               revenue_resid = revenue - price_hajek_estimator_95[[1]]) 
        reg_data_eff_95 <- rbind(reg_data_eff_95_control, reg_data_eff_95_treatment)
    
        indices_95 <- c(reg_data_eff_95_treatment$rn, reg_data_eff_95_control$rn)
        
        hajek_var_95_bookings <- ht_var(hajek_prob_matrices[[3]][[1]],
                                        hajek_prob_matrices[[3]][[2]],
                                        hajek_prob_matrices[[3]][[3]],
                                        reg_data_eff_95, reg_data_eff_95_control, reg_data_eff_95_treatment,
                                        reg_data_eff_95_treatment$rn, reg_data_eff_95_control$rn, indices_95,
                                        'booked_resid', 'eff_treat_95')
        
        hajek_var_95_price <- ht_var(hajek_prob_matrices[[3]][[1]],
                                     hajek_prob_matrices[[3]][[2]],
                                     hajek_prob_matrices[[3]][[3]],
                                     reg_data_eff_95, reg_data_eff_95_control, reg_data_eff_95_treatment,
                                     reg_data_eff_95_treatment$rn, reg_data_eff_95_control$rn, indices_95,
                                     'revenue_resid', 'eff_treat_95')
    }
    
    regression_bookings <- felm(booked ~ assignment | 0 | 0 | cluster_membership, data = reg_data)
    regression_price <- felm(revenue ~ assignment | 0 | 0 | cluster_membership, data = reg_data)
    
    regression_bookings_coef <- coef(regression_bookings)[2]
    regression_price_coef <- coef(regression_price)[2]
    regression_bookings_var <- diag(regression_bookings$clustervcv)[2]
    regression_price_var <- diag(regression_price$clustervcv)[2]     
    
    if(hajek_estimator_50 == 'NA') {
        hajek_estimator_50_report <- 'NA'
        price_hajek_estimator_50_report <- 'NA'
    } else {
        hajek_estimator_50_report <- hajek_estimator_50[[1]] - hajek_estimator_50[[2]]
        price_hajek_estimator_50_report <- price_hajek_estimator_50[[1]] - price_hajek_estimator_50[[2]]
    }

    if(hajek_estimator_75 == 'NA') {
        hajek_estimator_75_report <- 'NA'
        price_hajek_estimator_75_report <- 'NA'
    } else {
        hajek_estimator_75_report <- hajek_estimator_75[[1]] - hajek_estimator_75[[2]]
        price_hajek_estimator_75_report <- price_hajek_estimator_75[[1]] - price_hajek_estimator_75[[2]]
    }    
    
    if(hajek_estimator_95 == 'NA') {
        hajek_estimator_95_report <- 'NA'
        price_hajek_estimator_95_report <- 'NA'
    } else {
        hajek_estimator_95_report <- hajek_estimator_95[[1]] - hajek_estimator_95[[2]]
        price_hajek_estimator_95_report <- price_hajek_estimator_95[[1]] - price_hajek_estimator_95[[2]]
    }      
    
    sink()
    unlink(paste0(
         c(data_directory, i, '.txt'),
         sep='', collapse=''
     ))
  
    data.frame(treatment_booked = treatment_booked, 
               treatment_unbooked = treatment_unbooked, 
               treatment_booking_rate = treatment_booked/(treatment_unbooked + treatment_booked),
               treatment_revenue = treatment_revenue,
               treatment_revenue_per_user = treatment_revenue/(treatment_unbooked + treatment_booked),
               control_booked = control_booked,
               control_unbooked = control_unbooked,
               control_booking_rate = control_booked/(control_unbooked + control_booked),
               control_revenue = control_revenue,
               control_revenue_per_user = control_revenue/(control_unbooked + control_booked),
               p_value = p_value,
               eff_treatment_booked_75 = eff_treatment_booked_75, 
               eff_treatment_unbooked_75 = eff_treatment_unbooked_75, 
               eff_treatment_booking_rate_75 = eff_treatment_booked_75/(eff_treatment_unbooked_75 + 
                                                                        eff_treatment_booked_75),
               eff_treatment_revenue_75 = eff_treatment_revenue_75,
               eff_treatment_revenue_per_user_75 = eff_treatment_revenue_75/(eff_treatment_unbooked_75 + 
                                                                             eff_treatment_booked_75),
               eff_control_booked_75 = eff_control_booked_75,
               eff_control_unbooked_75 = eff_control_unbooked_75,
               eff_control_booking_rate_75 = eff_control_booked_75/(eff_control_unbooked_75 + 
                                                                    eff_control_booked_75),
               eff_control_revenue_75 = eff_control_revenue_75,
               eff_control_revenue_per_user_75 = eff_control_revenue_75/(eff_control_unbooked_75 + 
                                                                         eff_control_booked_75),
               eff_p_value_75 = eff_p_value_75,
               eff_treatment_booked_50 = eff_treatment_booked_50, 
               eff_treatment_unbooked_50 = eff_treatment_unbooked_50, 
               eff_treatment_booking_rate_50 = eff_treatment_booked_50/(eff_treatment_unbooked_50 + 
                                                                        eff_treatment_booked_50),
               eff_treatment_revenue_50 = eff_treatment_revenue_50,
               eff_treatment_revenue_per_user_50 = eff_treatment_revenue_50/(eff_treatment_unbooked_50 + 
                                                                             eff_treatment_booked_50),
               eff_control_booked_50 = eff_control_booked_50,
               eff_control_unbooked_50 = eff_control_unbooked_50,
               eff_control_booking_rate_50 = eff_control_booked_50/(eff_control_unbooked_50 + 
                                                                    eff_control_booked_50),
               eff_control_revenue_50 = eff_control_revenue_50,
               eff_control_revenue_per_user_50 = eff_control_revenue_50/(eff_control_unbooked_50 + 
                                                                         eff_control_booked_50),
               eff_p_value_50 = eff_p_value_50,
               eff_treatment_booked_95 = eff_treatment_booked_95, 
               eff_treatment_unbooked_95 = eff_treatment_unbooked_95, 
               eff_treatment_booking_rate_95 = eff_treatment_booked_95/(eff_treatment_unbooked_95 + 
                                                                        eff_treatment_booked_95),
               eff_treatment_revenue_95 = eff_treatment_revenue_95,
               eff_treatment_revenue_per_user_95 = eff_treatment_revenue_95/(eff_treatment_unbooked_95 + 
                                                                             eff_treatment_booked_95),
               eff_control_booked_95 = eff_control_booked_95,
               eff_control_unbooked_95 = eff_control_unbooked_95,
               eff_control_booking_rate_95 = eff_control_booked_95/(eff_control_unbooked_95 + 
                                                                    eff_control_booked_95),
               eff_control_revenue_95 = eff_control_revenue_95,
               eff_control_revenue_per_user_95 = eff_control_revenue_95/(eff_control_unbooked_95 + 
                                                                         eff_control_booked_95),
               eff_p_value_95 = eff_p_value_95,
               hajek_estimator_75 = hajek_estimator_75_report,
               hajek_estimator_95 = hajek_estimator_95_report,
               hajek_estimator_50 = hajek_estimator_50_report,
               price_hajek_estimator_75 = price_hajek_estimator_75_report,
               price_hajek_estimator_95 = price_hajek_estimator_95_report,
               price_hajek_estimator_50 = price_hajek_estimator_50_report,
               price_p_value = price_p_value,
               eff_price_p_value_50 = eff_price_p_value_50,
               eff_price_p_value_75 = eff_price_p_value_75,
               eff_price_p_value_95 = eff_price_p_value_95,
               booking_diff_var = booking_diff_var,
               price_diff_var = price_diff_var,
               booking_diff_50_var = booking_diff_50_var,
               price_diff_50_var = price_diff_50_var,
               booking_diff_75_var = booking_diff_75_var,
               price_diff_75_var = price_diff_75_var,
               booking_diff_95_var = booking_diff_95_var,
               price_diff_95_var = price_diff_95_var, 
               hajek_var_50_bookings = hajek_var_50_bookings,
               hajek_var_50_price = hajek_var_50_price,
               hajek_var_75_bookings = hajek_var_75_bookings,
               hajek_var_75_price = hajek_var_75_price,
               hajek_var_95_bookings = hajek_var_95_bookings,
               hajek_var_95_price = hajek_var_95_price,
               regression_bookings_coef = regression_bookings_coef,
               regression_price_coef = regression_price_coef,
               regression_bookings_var = regression_bookings_var,
               regression_price_var = regression_price_var
  )
}

In [9]:
simulate_graph_randomization_wash <- function(data, pct, effect_type, effect_size, sim_info,
                                      n_searchers, sim_info_index, consumer_type, ev,
                                        community, graph, block,
                                        prob_vars,
                                        hajek_prob_matrices) {
    
    sink(paste0(
         c(data_directory, i,'.txt'),
         sep='', collapse=''
     ))

    n_communities <- length(unique(community))
    assignments <- rep(0, n_communities)
    assignments[as.numeric(
        as.character(unlist(blockTools::assignment(block)[[1]][[1]]['Treatment 1'])))] <- 1
    vertsInAssignments <- community %in% which(assignments == 1)
  
    data$cluster_membership <- community
    data <- data %>% mutate(assignment = ifelse(assignments[cluster_membership] == 1,
                                       1, 0))
    
    if(effect_type == 'price') {
        data <- data %>% 
        mutate(price_scaled = as.numeric(scale(price, scale=TRUE))) %>%
        mutate(price_scaled = ifelse(assignment == 1, price_scaled-effect_size, price_scaled)) 
    } else{
        data <- data %>% 
        mutate(unobserved_component = ifelse(assignment == 1, 
                                             unobserved_component+effect_size, 
                                             unobserved_component))
    }
    
    data %>% 
    mutate(search_quality = sim_info[[sim_info_index]]$search_weights[1]*reviews_scaled + 
           sim_info[[sim_info_index]]$search_weights[2]*satisfaction_scaled + 
           sim_info[[sim_info_index]]$search_weights[3]*bed_scaled + 
           sim_info[[sim_info_index]]$search_weights[5]*minstay_scaled -
           sim_info[[sim_info_index]]$search_weights[6]*price_scaled + 
           sim_info[[sim_info_index]]$search_weights[7]*`Entire home/apt` + 
           sim_info[[sim_info_index]]$search_weights[8]*`Private room` + 
           sim_info[[sim_info_index]]$search_weights[9]*`Shared room`) -> data
    
    data$pct_treated_neighbors <- sapply(1:vcount(graph), FUN=function(i){
        neigh <- neighbors(graph, i)
        sum(vertsInAssignments[neigh])/length(neigh)
    })
    
    min_search = sim_info[[sim_info_index]]$min_search_acc
  
    booked_rooms <- filter(data, room_id == -1)
  
  
    for (j in 1:n_searchers) {
        
        min_search_lon <- min(sim_info[[sim_info_index]]$lon_search_bounds[j,])
        max_search_lon <- max(sim_info[[sim_info_index]]$lon_search_bounds[j,])
        min_search_lat <- min(sim_info[[sim_info_index]]$lat_search_bounds[j,])
        max_search_lat <- max(sim_info[[sim_info_index]]$lat_search_bounds[j,])
        min_search_acc <- floor(min_search[j])
        
        if(consumer_type == 'uniform') {
            searcher_weights <- sim_info[[sim_info_index]]$unif_searcher_weights[,j]
        } else {
            searcher_weights <- sim_info[[sim_info_index]]$user_preferences_normal[j,]
        }
        
        if(consumer_type != 'uniform') {
            data$ev_error <- extreme_value_distribution[j,][data$rn]
            income <- sim_info[[sim_info_index]]$income[j]
            
            data %>% 
            filter(latitude >= min_search_lat & latitude <= max_search_lat &
                   longitude >= min_search_lon & longitude <= max_search_lon & 
                   accommodates >= min_search_acc) %>%
            arrange(desc(search_quality)) %>% 
            head(n=10) %>%
            mutate(searcher_quality = searcher_weights[1]*reviews_scaled + 
                   searcher_weights[2]*satisfaction_scaled + 
                   searcher_weights[3]*bed_scaled - 
                   searcher_weights[5]*minstay_scaled +
                   abs(searcher_weights[6])*(income-price_scaled) + 
                   searcher_weights[7]*`Entire home/apt` +
                   searcher_weights[8]*`Private room` + 
                   searcher_weights[9]*`Shared room` + 
                   unobserved_component + 
                   ev_error) %>% 
            arrange(desc(searcher_quality)) %>% 
            head(n=1) -> searcher_selection
            
            if(nrow(searcher_selection) == 1 & searcher_selection$searcher_quality[1] >= 0) {
                booked_rooms <- rbind(booked_rooms, searcher_selection)
                data <- filter(data, room_id != searcher_selection$room_id[1])
                }
            
            } else {
            
            search_quality_cutoff <- sim_info[[sim_info_index]]$search_quality_cutoff[j]
            data %>% 
            filter(latitude >= min_search_lat & latitude <= max_search_lat &
                   longitude >= min_search_lon & longitude <= max_search_lon & 
                   accommodates >= min_search_acc) %>%
            arrange(desc(search_quality)) %>% 
            head(n=10) %>%
            mutate(searcher_quality = searcher_weights[1]*reviews_scaled + 
                   searcher_weights[2]*satisfaction_scaled + 
                   searcher_weights[3]*bed_scaled - 
                   searcher_weights[5]*minstay_scaled - 
                   abs(searcher_weights[6])*price_scaled + 
                   searcher_weights[7]*`Entire home/apt` +
                   searcher_weights[8]*`Private room` + 
                   searcher_weights[9]*`Shared room`) %>% 
            arrange(desc(searcher_quality)) %>% 
            head(n=1) -> searcher_selection
            
            if(nrow(searcher_selection) == 1 & searcher_selection$searcher_quality[1] >= search_quality_cutoff) {
                booked_rooms <- rbind(booked_rooms, searcher_selection)
                data <- filter(data, room_id != searcher_selection$room_id[1])
            }
        }
    }
    
    data <- data %>% 
      mutate(eff_treat_50 = ifelse(assignment == 1 & pct_treated_neighbors >= .5, 1, 0),
             eff_treat_75 = ifelse(assignment == 1 & pct_treated_neighbors >= .75, 1, 0),
             eff_treat_95 = ifelse(assignment == 1 & pct_treated_neighbors >= .95, 1, 0),
             eff_control_50 = ifelse(assignment == 0 & (1-pct_treated_neighbors) >= .5, 1, 0),
             eff_control_75 = ifelse(assignment == 0 & (1-pct_treated_neighbors) >= .75, 1, 0),
             eff_control_95 = ifelse(assignment == 0 & (1-pct_treated_neighbors) >= .95, 1, 0),
             revenue = 0,
             booked = 0)
    
    booked_rooms <- booked_rooms %>% 
      mutate(eff_treat_50 = ifelse(assignment == 1 & pct_treated_neighbors >= .5, 1, 0),
             eff_treat_75 = ifelse(assignment == 1 & pct_treated_neighbors >= .75, 1, 0),
             eff_treat_95 = ifelse(assignment == 1 & pct_treated_neighbors >= .95, 1, 0),
             eff_control_50 = ifelse(assignment == 0 & (1-pct_treated_neighbors) >= .5, 1, 0),
             eff_control_75 = ifelse(assignment == 0 & (1-pct_treated_neighbors) >= .75, 1, 0),
             eff_control_95 = ifelse(assignment == 0 & (1-pct_treated_neighbors) >= .95, 1, 0),
             revenue = price,
             booked = 1)
    
  
    treatment_booked <- nrow(filter(booked_rooms, assignment == 1))
    treatment_unbooked <- nrow(filter(data, assignment == 1))
    treatment_revenue <- sum(filter(booked_rooms, assignment == 1)$price)
    control_booked <- nrow(filter(booked_rooms, assignment == 0))
    control_unbooked <- nrow(filter(data, assignment == 0))
    control_revenue <- sum(filter(booked_rooms, assignment == 0)$price)
    
    eff_treatment_booked_50 <- nrow(filter(booked_rooms, eff_treat_50 == 1))
    eff_treatment_unbooked_50 <- nrow(filter(data, eff_treat_50 == 1))
    eff_treatment_revenue_50 <- sum(filter(booked_rooms, eff_treat_50 == 1)$price)
    eff_control_booked_50 <- nrow(filter(booked_rooms, eff_control_50 == 1))
    eff_control_unbooked_50 <- nrow(filter(data, eff_control_50 == 1))
    eff_control_revenue_50 <- sum(filter(booked_rooms, eff_control_50 == 1)$price)
  
    eff_treatment_booked_75 <- nrow(filter(booked_rooms, eff_treat_75 == 1))
    eff_treatment_unbooked_75 <- nrow(filter(data, eff_treat_75 == 1))
    eff_treatment_revenue_75 <- sum(filter(booked_rooms, eff_treat_75 == 1)$price)
    eff_control_booked_75 <- nrow(filter(booked_rooms, eff_control_75 == 1))
    eff_control_unbooked_75 <- nrow(filter(data, eff_control_75 == 1))
    eff_control_revenue_75 <- sum(filter(booked_rooms, eff_control_75 == 1)$price)
  
    eff_treatment_booked_95 <- nrow(filter(booked_rooms, eff_treat_95 == 1))
    eff_treatment_unbooked_95 <- nrow(filter(data, eff_treat_95 == 1))
    eff_treatment_revenue_95 <- sum(filter(booked_rooms, eff_treat_95 == 1)$price)
    eff_control_booked_95 <- nrow(filter(booked_rooms, eff_control_95 == 1))
    eff_control_unbooked_95 <- nrow(filter(data, eff_control_95 == 1))
    eff_control_revenue_95 <- sum(filter(booked_rooms, eff_control_95 == 1)$price)
    
    treatment_prices <- c(filter(booked_rooms, assignment == 1)$price, rep(0, treatment_unbooked))
    eff_treatment_prices_50 <- c(filter(booked_rooms, eff_treat_50 == 1)$price, rep(0, eff_treatment_unbooked_50))
    eff_treatment_prices_75 <- c(filter(booked_rooms, eff_treat_75 == 1)$price, rep(0, eff_treatment_unbooked_75))
    eff_treatment_prices_95 <- c(filter(booked_rooms, eff_treat_95 == 1)$price, rep(0, eff_treatment_unbooked_95))
    
    control_prices <- c(filter(booked_rooms, assignment == 0)$price, rep(0, control_unbooked))
    eff_control_prices_50 <- c(filter(booked_rooms, eff_control_50 == 1)$price, rep(0, eff_control_unbooked_50))
    eff_control_prices_75 <- c(filter(booked_rooms, eff_control_75 == 1)$price, rep(0, eff_control_unbooked_75))
    eff_control_prices_95 <- c(filter(booked_rooms, eff_control_95 == 1)$price, rep(0, eff_control_unbooked_95)) 
    
    reg_data <- rbind(data %>% 
                      dplyr::select(rn, booked, revenue, assignment, cluster_membership,
                                    eff_control_50, eff_treat_50, eff_control_75, eff_treat_75,
                                    eff_control_95, eff_treat_95),
                      booked_rooms %>%
                      dplyr::select(rn, booked, revenue, assignment, cluster_membership,
                                    eff_control_50, eff_treat_50, eff_control_75, eff_treat_75,
                                    eff_control_95, eff_treat_95)) %>% arrange(rn)
    
    booking_diff_var <- treatment_var_bookings(control_booked, control_unbooked,
                                               treatment_booked, treatment_unbooked)
    price_diff_var <- var(treatment_prices)/length(treatment_prices) + var(control_prices)/length(control_prices)
    
    booking_diff_50_var <- treatment_var_bookings(eff_control_booked_50, eff_control_unbooked_50,
                                               eff_treatment_booked_50, eff_treatment_unbooked_50)
    price_diff_50_var <- var(eff_treatment_prices_50)/length(eff_treatment_prices_50) + 
                         var(eff_control_prices_50)/length(eff_control_prices_50)
    
    booking_diff_75_var <- treatment_var_bookings(eff_control_booked_75, eff_control_unbooked_75,
                                               eff_treatment_booked_75, eff_treatment_unbooked_75)
    price_diff_75_var <- var(eff_treatment_prices_75)/length(eff_treatment_prices_75) + 
                         var(eff_control_prices_75)/length(eff_control_prices_75)

    booking_diff_95_var <- treatment_var_bookings(eff_control_booked_95, eff_control_unbooked_95,
                                               eff_treatment_booked_95, eff_treatment_unbooked_95)
    price_diff_95_var <- var(eff_treatment_prices_95)/length(eff_treatment_prices_95) + 
                         var(eff_control_prices_95)/length(eff_control_prices_95)
    
    p_value <- ifelse(pct %in% c(0,1) | (treatment_booked + treatment_unbooked) <= 0 | 
                      (control_booked + control_unbooked) <= 0, NA, 
                      prop.test(c(treatment_booked, control_booked), 
                                c(treatment_booked + treatment_unbooked, 
                                  control_booked + control_unbooked))$p.value)
    
    if(pct %in% c(0,1) | (eff_treatment_booked_75 + eff_treatment_unbooked_75) <= 0 | 
                             (eff_control_booked_75 + eff_control_unbooked_75) <= 0) {
        eff_p_value_75 <- 'NA'
    } else {
        eff_p_value_75 <- prop.test(c(eff_treatment_booked_75, eff_control_booked_75), 
                                           c(eff_treatment_booked_75 + eff_treatment_unbooked_75, 
                                             eff_control_booked_75 + eff_control_unbooked_75))$p.value
    }
    
    if(pct %in% c(0,1) | (eff_treatment_booked_50 + eff_treatment_unbooked_50) <= 0 | 
                             (eff_control_booked_50 + eff_control_unbooked_50) <= 0) {
        eff_p_value_50 <- 'NA'
    } else {
        eff_p_value_50 <- prop.test(c(eff_treatment_booked_50, eff_control_booked_50), 
                                       c(eff_treatment_booked_50 + eff_treatment_unbooked_50, 
                                         eff_control_booked_50 + eff_control_unbooked_50))$p.value
    }
    
    if(pct %in% c(0,1) | (eff_treatment_booked_95 + eff_treatment_unbooked_95) <= 0 |
                             (eff_control_booked_95 + eff_control_unbooked_95) <= 0) {
        eff_p_value_95 <- 'NA'
    } else{
        eff_p_value_95 <- prop.test(c(eff_treatment_booked_95, eff_control_booked_95), 
                                       c(eff_treatment_booked_95 + eff_treatment_unbooked_95, 
                                         eff_control_booked_95 + eff_control_unbooked_95))$p.value
    }
    
    price_p_value <- ifelse(pct %in% c(0,1), 'NA', wilcox.test(treatment_prices, control_prices)$p.value)
    
    if(pct %in% c(0,1) | length(eff_treatment_prices_50) <= 1 | length(eff_control_prices_50) <= 1) {
        eff_price_p_value_50 <- 'NA'
    } else {
        eff_price_p_value_50 <- wilcox.test(eff_treatment_prices_50, eff_control_prices_50)$p.value
    }
    
    if(pct %in% c(0,1) | length(eff_treatment_prices_75) <= 1 | length(eff_control_prices_75) <= 1) {
        eff_price_p_value_75 <- 'NA'
    } else {
        eff_price_p_value_75 <- wilcox.test(eff_treatment_prices_75, eff_control_prices_75)$p.value
    }
    
    if(pct %in% c(0,1) | length(eff_treatment_prices_95) <= 1 | length(eff_control_prices_95) <= 1) {
        eff_price_p_value_95 <- 'NA'
    } else {
        eff_price_p_value_95 <- wilcox.test(eff_treatment_prices_95, eff_control_prices_95)$p.value
    }
    
    if(length(eff_treatment_prices_50) <= 1 | length(eff_control_prices_50) <= 1) {
        hajek_estimator_50 <- 'NA'
        price_hajek_estimator_50 <- 'NA'
        } else {
        hajek_estimator_50 <- hajek_estimator(booked_rooms, data, prob_vars[1], prob_vars[2], .5, 'booked')
        price_hajek_estimator_50 <- hajek_estimator(booked_rooms, data, prob_vars[1], prob_vars[2], .5, 'revenue')
    }

    if(length(eff_treatment_prices_75) <= 1 | length(eff_control_prices_75) <= 1) {
        hajek_estimator_75 <- 'NA'
        price_hajek_estimator_75 <- 'NA'
        } else {
        hajek_estimator_75 <- hajek_estimator(booked_rooms, data, prob_vars[3], prob_vars[4], .75, 'booked')
        price_hajek_estimator_75 <- hajek_estimator(booked_rooms, data, prob_vars[3], prob_vars[4], .75, 'revenue')
    }    

    if(length(eff_treatment_prices_95) <= 1 | length(eff_control_prices_95) <= 1) {
        hajek_estimator_95 <- 'NA'
        price_hajek_estimator_95 <- 'NA'
        } else {
        hajek_estimator_95 <- hajek_estimator(booked_rooms, data, prob_vars[5], prob_vars[6], .95, 'booked')
        price_hajek_estimator_95 <- hajek_estimator(booked_rooms, data, prob_vars[5], prob_vars[6], .95, 'revenue') 
    }    
    
    if(length(eff_treatment_prices_50) <= 1 | length(eff_control_prices_50) <= 1) {
        hajek_var_50_bookings <- 'NA'
        hajek_var_50_price <- 'NA'
        } else {
        reg_data_eff_50_control <- filter(reg_data, eff_control_50 == 1) %>% 
        mutate(booked_resid = booked - hajek_estimator_50[[2]],
               revenue_resid = revenue - price_hajek_estimator_50[[2]])
        reg_data_eff_50_treatment <- filter(reg_data, eff_treat_50 == 1) %>% 
        mutate(booked_resid = booked - hajek_estimator_50[[1]],
               revenue_resid = revenue - price_hajek_estimator_50[[1]])
        reg_data_eff_50 <- rbind(reg_data_eff_50_control, reg_data_eff_50_treatment)
        
        indices_50 <- c(reg_data_eff_50_treatment$rn, reg_data_eff_50_control$rn)
        
        hajek_var_50_bookings <- ht_var(hajek_prob_matrices[[1]][[1]],
                                        hajek_prob_matrices[[1]][[2]],
                                        hajek_prob_matrices[[1]][[3]],
                                        reg_data_eff_50, reg_data_eff_50_control, reg_data_eff_50_treatment,
                                        reg_data_eff_50_treatment$rn, reg_data_eff_50_control$rn, indices_50,
                                        'booked_resid', 'eff_treat_50')
        
        hajek_var_50_price <- ht_var(hajek_prob_matrices[[1]][[1]],
                                     hajek_prob_matrices[[1]][[2]],
                                     hajek_prob_matrices[[1]][[3]],
                                     reg_data_eff_50, reg_data_eff_50_control, reg_data_eff_50_treatment,
                                     reg_data_eff_50_treatment$rn, reg_data_eff_50_control$rn, indices_50,
                                     'revenue_resid', 'eff_treat_50')
    }
    
    if(length(eff_treatment_prices_75) <= 1 | length(eff_control_prices_75) <= 1) {
        hajek_var_75_bookings <- 'NA'
        hajek_var_75_price <- 'NA'
        } else {
        
        reg_data_eff_75_control <- filter(reg_data, eff_control_75 == 1) %>% 
        mutate(booked_resid = booked - hajek_estimator_75[[2]],
               revenue_resid = revenue - price_hajek_estimator_75[[2]]) 
        reg_data_eff_75_treatment <- filter(reg_data, eff_treat_75 == 1) %>% 
        mutate(booked_resid = booked - hajek_estimator_75[[1]],
               revenue_resid = revenue - price_hajek_estimator_75[[1]]) 
        reg_data_eff_75 <- rbind(reg_data_eff_75_control, reg_data_eff_75_treatment)
        
        indices_75 <- c(reg_data_eff_75_treatment$rn, reg_data_eff_75_control$rn)
        
        hajek_var_75_bookings <- ht_var(hajek_prob_matrices[[2]][[1]],
                                        hajek_prob_matrices[[2]][[2]],
                                        hajek_prob_matrices[[2]][[3]],
                                        reg_data_eff_75, reg_data_eff_75_control, reg_data_eff_75_treatment,
                                        reg_data_eff_75_treatment$rn, reg_data_eff_75_control$rn, indices_75,
                                        'booked_resid', 'eff_treat_75')
        
        hajek_var_75_price <- ht_var(hajek_prob_matrices[[2]][[1]],
                                     hajek_prob_matrices[[2]][[2]],
                                     hajek_prob_matrices[[2]][[3]],
                                     reg_data_eff_75, reg_data_eff_75_control, reg_data_eff_75_treatment,
                                     reg_data_eff_75_treatment$rn, reg_data_eff_75_control$rn, indices_75,
                                     'revenue_resid', 'eff_treat_75')
    }
    
    if(length(eff_treatment_prices_95) <= 1 | length(eff_control_prices_95) <= 1) {
        hajek_var_95_bookings <- 'NA'
        hajek_var_95_price <- 'NA'
        } else {
        
        reg_data_eff_95_control <- filter(reg_data, eff_control_95 == 1) %>% 
        mutate(booked_resid = booked - hajek_estimator_95[[2]],
               revenue_resid = revenue - price_hajek_estimator_95[[2]]) 
        reg_data_eff_95_treatment <- filter(reg_data, eff_treat_95 == 1) %>% 
        mutate(booked_resid = booked - hajek_estimator_95[[1]],
               revenue_resid = revenue - price_hajek_estimator_95[[1]]) 
        reg_data_eff_95 <- rbind(reg_data_eff_95_control, reg_data_eff_95_treatment)
    
        indices_95 <- c(reg_data_eff_95_treatment$rn, reg_data_eff_95_control$rn)
        
        hajek_var_95_bookings <- ht_var(hajek_prob_matrices[[3]][[1]],
                                        hajek_prob_matrices[[3]][[2]],
                                        hajek_prob_matrices[[3]][[3]],
                                        reg_data_eff_95, reg_data_eff_95_control, reg_data_eff_95_treatment,
                                        reg_data_eff_95_treatment$rn, reg_data_eff_95_control$rn, indices_95,
                                        'booked_resid', 'eff_treat_95')
        
        hajek_var_95_price <- ht_var(hajek_prob_matrices[[3]][[1]],
                                     hajek_prob_matrices[[3]][[2]],
                                     hajek_prob_matrices[[3]][[3]],
                                     reg_data_eff_95, reg_data_eff_95_control, reg_data_eff_95_treatment,
                                     reg_data_eff_95_treatment$rn, reg_data_eff_95_control$rn, indices_95,
                                     'revenue_resid', 'eff_treat_95')
    }
    
    regression_bookings <- felm(booked ~ assignment | 0 | 0 | cluster_membership, data = reg_data)
    regression_price <- felm(revenue ~ assignment | 0 | 0 | cluster_membership, data = reg_data)
    
    regression_bookings_coef <- coef(regression_bookings)[2]
    regression_price_coef <- coef(regression_price)[2]
    regression_bookings_var <- diag(regression_bookings$clustervcv)[2]
    regression_price_var <- diag(regression_price$clustervcv)[2]     
    
    if(hajek_estimator_50 == 'NA') {
        hajek_estimator_50_report <- 'NA'
        price_hajek_estimator_50_report <- 'NA'
    } else {
        hajek_estimator_50_report <- hajek_estimator_50[[1]] - hajek_estimator_50[[2]]
        price_hajek_estimator_50_report <- price_hajek_estimator_50[[1]] - price_hajek_estimator_50[[2]]
    }

    if(hajek_estimator_75 == 'NA') {
        hajek_estimator_75_report <- 'NA'
        price_hajek_estimator_75_report <- 'NA'
    } else {
        hajek_estimator_75_report <- hajek_estimator_75[[1]] - hajek_estimator_75[[2]]
        price_hajek_estimator_75_report <- price_hajek_estimator_75[[1]] - price_hajek_estimator_75[[2]]
    }    
    
    if(hajek_estimator_95 == 'NA') {
        hajek_estimator_95_report <- 'NA'
        price_hajek_estimator_95_report <- 'NA'
    } else {
        hajek_estimator_95_report <- hajek_estimator_95[[1]] - hajek_estimator_95[[2]]
        price_hajek_estimator_95_report <- price_hajek_estimator_95[[1]] - price_hajek_estimator_95[[2]]
    }      
    
    sink()
    unlink(paste0(
         c(data_directory, i, '.txt'),
         sep='', collapse=''
     ))
  
    data.frame(treatment_booked = treatment_booked, 
               treatment_unbooked = treatment_unbooked, 
               treatment_booking_rate = treatment_booked/(treatment_unbooked + treatment_booked),
               treatment_revenue = treatment_revenue,
               treatment_revenue_per_user = treatment_revenue/(treatment_unbooked + treatment_booked),
               control_booked = control_booked,
               control_unbooked = control_unbooked,
               control_booking_rate = control_booked/(control_unbooked + control_booked),
               control_revenue = control_revenue,
               control_revenue_per_user = control_revenue/(control_unbooked + control_booked),
               p_value = p_value,
               eff_treatment_booked_75 = eff_treatment_booked_75, 
               eff_treatment_unbooked_75 = eff_treatment_unbooked_75, 
               eff_treatment_booking_rate_75 = eff_treatment_booked_75/(eff_treatment_unbooked_75 + 
                                                                        eff_treatment_booked_75),
               eff_treatment_revenue_75 = eff_treatment_revenue_75,
               eff_treatment_revenue_per_user_75 = eff_treatment_revenue_75/(eff_treatment_unbooked_75 + 
                                                                             eff_treatment_booked_75),
               eff_control_booked_75 = eff_control_booked_75,
               eff_control_unbooked_75 = eff_control_unbooked_75,
               eff_control_booking_rate_75 = eff_control_booked_75/(eff_control_unbooked_75 + 
                                                                    eff_control_booked_75),
               eff_control_revenue_75 = eff_control_revenue_75,
               eff_control_revenue_per_user_75 = eff_control_revenue_75/(eff_control_unbooked_75 + 
                                                                         eff_control_booked_75),
               eff_p_value_75 = eff_p_value_75,
               eff_treatment_booked_50 = eff_treatment_booked_50, 
               eff_treatment_unbooked_50 = eff_treatment_unbooked_50, 
               eff_treatment_booking_rate_50 = eff_treatment_booked_50/(eff_treatment_unbooked_50 + 
                                                                        eff_treatment_booked_50),
               eff_treatment_revenue_50 = eff_treatment_revenue_50,
               eff_treatment_revenue_per_user_50 = eff_treatment_revenue_50/(eff_treatment_unbooked_50 + 
                                                                             eff_treatment_booked_50),
               eff_control_booked_50 = eff_control_booked_50,
               eff_control_unbooked_50 = eff_control_unbooked_50,
               eff_control_booking_rate_50 = eff_control_booked_50/(eff_control_unbooked_50 + 
                                                                    eff_control_booked_50),
               eff_control_revenue_50 = eff_control_revenue_50,
               eff_control_revenue_per_user_50 = eff_control_revenue_50/(eff_control_unbooked_50 + 
                                                                         eff_control_booked_50),
               eff_p_value_50 = eff_p_value_50,
               eff_treatment_booked_95 = eff_treatment_booked_95, 
               eff_treatment_unbooked_95 = eff_treatment_unbooked_95, 
               eff_treatment_booking_rate_95 = eff_treatment_booked_95/(eff_treatment_unbooked_95 + 
                                                                        eff_treatment_booked_95),
               eff_treatment_revenue_95 = eff_treatment_revenue_95,
               eff_treatment_revenue_per_user_95 = eff_treatment_revenue_95/(eff_treatment_unbooked_95 + 
                                                                             eff_treatment_booked_95),
               eff_control_booked_95 = eff_control_booked_95,
               eff_control_unbooked_95 = eff_control_unbooked_95,
               eff_control_booking_rate_95 = eff_control_booked_95/(eff_control_unbooked_95 + 
                                                                    eff_control_booked_95),
               eff_control_revenue_95 = eff_control_revenue_95,
               eff_control_revenue_per_user_95 = eff_control_revenue_95/(eff_control_unbooked_95 + 
                                                                         eff_control_booked_95),
               eff_p_value_95 = eff_p_value_95,
               hajek_estimator_75 = hajek_estimator_75_report,
               hajek_estimator_95 = hajek_estimator_95_report,
               hajek_estimator_50 = hajek_estimator_50_report,
               price_hajek_estimator_75 = price_hajek_estimator_75_report,
               price_hajek_estimator_95 = price_hajek_estimator_95_report,
               price_hajek_estimator_50 = price_hajek_estimator_50_report,
               price_p_value = price_p_value,
               eff_price_p_value_50 = eff_price_p_value_50,
               eff_price_p_value_75 = eff_price_p_value_75,
               eff_price_p_value_95 = eff_price_p_value_95,
               booking_diff_var = booking_diff_var,
               price_diff_var = price_diff_var,
               booking_diff_50_var = booking_diff_50_var,
               price_diff_50_var = price_diff_50_var,
               booking_diff_75_var = booking_diff_75_var,
               price_diff_75_var = price_diff_75_var,
               booking_diff_95_var = booking_diff_95_var,
               price_diff_95_var = price_diff_95_var, 
               hajek_var_50_bookings = hajek_var_50_bookings,
               hajek_var_50_price = hajek_var_50_price,
               hajek_var_75_bookings = hajek_var_75_bookings,
               hajek_var_75_price = hajek_var_75_price,
               hajek_var_95_bookings = hajek_var_95_bookings,
               hajek_var_95_price = hajek_var_95_price,
               regression_bookings_coef = regression_bookings_coef,
               regression_price_coef = regression_price_coef,
               regression_bookings_var = regression_bookings_var,
               regression_price_var = regression_price_var
  )
}

In [ ]:
#Establish baselines
cl <- makeSOCKcluster(n_clusters)
registerDoSNOW(cl)
treatment_reality_un_250 <- foreach(i = 1:n_simulations, 
                                    .combine = rbind,
                                    .multicombine= TRUE,
                                    .packages = c('dplyr', 'Matrix'),
                                    .maxcombine = 2) %dopar% simulate_ind_randomization(airbnb_listings, 
                                                                                  1,
                                                                                  'unobserved',
                                                                                  .75,
                                                                                  sim_info,
                                                                                  .25*max_n_searchers,
                                                                                  i,
                                                                                  'normal',
                                                                                        extreme_value_distribution)
stopCluster(cl)

cl <- makeSOCKcluster(n_clusters)
registerDoSNOW(cl)
treatment_reality_un_500 <- foreach(i = 1:n_simulations, 
                                    .combine = rbind,
                                    .multicombine= TRUE,
                                    .packages = c('dplyr', 'Matrix'),                                    
                                    .maxcombine = 2) %dopar% simulate_ind_randomization(airbnb_listings, 
                                                                                  1,
                                                                                  'unobserved',
                                                                                  .75,
                                                                                  sim_info,
                                                                                  .5*max_n_searchers,
                                                                                  i,
                                                                                  'normal',
                                                                                       extreme_value_distribution)
stopCluster(cl)

cl <- makeSOCKcluster(n_clusters)
registerDoSNOW(cl)
treatment_reality_un_1000 <- foreach(i = 1:n_simulations, 
                                    .combine = rbind,
                                    .multicombine= TRUE,
                                    .packages = c('dplyr', 'Matrix'),                                     
                                    .maxcombine = 2) %dopar% simulate_ind_randomization(airbnb_listings, 
                                                                                  1,
                                                                                  'unobserved',
                                                                                  .75,
                                                                                  sim_info,
                                                                                  max_n_searchers,
                                                                                  i,
                                                                                  'normal',
                                                                                       extreme_value_distribution)
stopCluster(cl)


cl <- makeSOCKcluster(n_clusters)
registerDoSNOW(cl)
control_reality_un_250 <- foreach(i = 1:n_simulations, 
                                    .combine = rbind,
                                    .multicombine= TRUE,
                                    .packages = c('dplyr', 'Matrix'),                                  
                                    .maxcombine = 2) %dopar% simulate_ind_randomization(airbnb_listings, 
                                                                                  0,
                                                                                  'unobserved',
                                                                                  .75,
                                                                                  sim_info,
                                                                                  .25*max_n_searchers,
                                                                                  i,
                                                                                  'normal',
                                                                                       extreme_value_distribution)
stopCluster(cl)

cl <- makeSOCKcluster(n_clusters)
registerDoSNOW(cl)
control_reality_un_500 <- foreach(i = 1:n_simulations, 
                                    .combine = rbind,
                                    .multicombine= TRUE,
                                    .packages = c('dplyr', 'Matrix'),                                  
                                    .maxcombine = 2) %dopar% simulate_ind_randomization(airbnb_listings, 
                                                                                  0,
                                                                                  'unobserved',
                                                                                  .75,
                                                                                  sim_info,
                                                                                  .5*max_n_searchers,
                                                                                  i,
                                                                                  'normal',
                                                                                       extreme_value_distribution)
stopCluster(cl)

cl <- makeSOCKcluster(n_clusters)
registerDoSNOW(cl)
control_reality_un_1000 <- foreach(i = 1:n_simulations, 
                                    .combine = rbind,
                                    .multicombine= TRUE,
                                    .packages = c('dplyr', 'Matrix'),                                   
                                    .maxcombine = 2) %dopar% simulate_ind_randomization(airbnb_listings, 
                                                                                  0,
                                                                                  'unobserved',
                                                                                  .75,
                                                                                  sim_info,
                                                                                  max_n_searchers,
                                                                                  i,
                                                                                  'normal',
                                                                                       extreme_value_distribution)
stopCluster(cl)

In [ ]:
#Establish baselines
cl <- makeSOCKcluster(n_clusters)
registerDoSNOW(cl)
treatment_reality_pn_250 <- foreach(i = 1:n_simulations, 
                                    .combine = rbind,
                                    .multicombine= TRUE,
                                    .packages = c('dplyr', 'Matrix'),
                                    .maxcombine = 2) %dopar% simulate_ind_randomization(airbnb_listings, 
                                                                                  1,
                                                                                  'price',
                                                                                  .75,
                                                                                  sim_info,
                                                                                  .25*max_n_searchers,
                                                                                  i,
                                                                                  'normal',
                                                                                        extreme_value_distribution)
stopCluster(cl)

cl <- makeSOCKcluster(n_clusters)
registerDoSNOW(cl)
treatment_reality_pn_500 <- foreach(i = 1:n_simulations, 
                                    .combine = rbind,
                                    .multicombine= TRUE,
                                    .packages = c('dplyr', 'Matrix'),                                    
                                    .maxcombine = 2) %dopar% simulate_ind_randomization(airbnb_listings, 
                                                                                  1,
                                                                                  'price',
                                                                                  .75,
                                                                                  sim_info,
                                                                                  .5*max_n_searchers,
                                                                                  i,
                                                                                  'normal',
                                                                                       extreme_value_distribution)
stopCluster(cl)

cl <- makeSOCKcluster(n_clusters)
registerDoSNOW(cl)
treatment_reality_pn_1000 <- foreach(i = 1:n_simulations, 
                                    .combine = rbind,
                                    .multicombine= TRUE,
                                    .packages = c('dplyr', 'Matrix'),                                     
                                    .maxcombine = 2) %dopar% simulate_ind_randomization(airbnb_listings, 
                                                                                  1,
                                                                                  'price',
                                                                                  .75,
                                                                                  sim_info,
                                                                                  max_n_searchers,
                                                                                  i,
                                                                                  'normal',
                                                                                       extreme_value_distribution)
stopCluster(cl)


cl <- makeSOCKcluster(n_clusters)
registerDoSNOW(cl)
control_reality_pn_250 <- foreach(i = 1:n_simulations, 
                                    .combine = rbind,
                                    .multicombine= TRUE,
                                    .packages = c('dplyr', 'Matrix'),                                  
                                    .maxcombine = 2) %dopar% simulate_ind_randomization(airbnb_listings, 
                                                                                  0,
                                                                                  'price',
                                                                                  .75,
                                                                                  sim_info,
                                                                                  .25*max_n_searchers,
                                                                                  i,
                                                                                  'normal',
                                                                                       extreme_value_distribution)
stopCluster(cl)

cl <- makeSOCKcluster(n_clusters)
registerDoSNOW(cl)
control_reality_pn_500 <- foreach(i = 1:n_simulations, 
                                    .combine = rbind,
                                    .multicombine= TRUE,
                                    .packages = c('dplyr', 'Matrix'),                                  
                                    .maxcombine = 2) %dopar% simulate_ind_randomization(airbnb_listings, 
                                                                                  0,
                                                                                  'price',
                                                                                  .75,
                                                                                  sim_info,
                                                                                  .5*max_n_searchers,
                                                                                  i,
                                                                                  'normal',
                                                                                       extreme_value_distribution)
stopCluster(cl)

cl <- makeSOCKcluster(n_clusters)
registerDoSNOW(cl)
control_reality_pn_1000 <- foreach(i = 1:n_simulations, 
                                    .combine = rbind,
                                    .multicombine= TRUE,
                                    .packages = c('dplyr', 'Matrix'),                                   
                                    .maxcombine = 2) %dopar% simulate_ind_randomization(airbnb_listings, 
                                                                                  0,
                                                                                  'price',
                                                                                  .75,
                                                                                  sim_info,
                                                                                  max_n_searchers,
                                                                                  i,
                                                                                  'normal',
                                                                                       extreme_value_distribution)
stopCluster(cl)

In [ ]:
#Establish baselines
cl <- makeSOCKcluster(n_clusters)
registerDoSNOW(cl)
treatment_reality_pu_250 <- foreach(i = 1:n_simulations, 
                                    .combine = rbind,
                                    .multicombine= TRUE,
                                    .packages = c('dplyr', 'Matrix'),
                                    .maxcombine = 2) %dopar% simulate_ind_randomization(airbnb_listings, 
                                                                                  1,
                                                                                  'price',
                                                                                  .75,
                                                                                  sim_info,
                                                                                  .25*max_n_searchers,
                                                                                  i,
                                                                                  'uniform',
                                                                                        extreme_value_distribution)
stopCluster(cl)

cl <- makeSOCKcluster(n_clusters)
registerDoSNOW(cl)
treatment_reality_pu_500 <- foreach(i = 1:n_simulations, 
                                    .combine = rbind,
                                    .multicombine= TRUE,
                                    .packages = c('dplyr', 'Matrix'),                                    
                                    .maxcombine = 2) %dopar% simulate_ind_randomization(airbnb_listings, 
                                                                                  1,
                                                                                  'price',
                                                                                  .75,
                                                                                  sim_info,
                                                                                  .5*max_n_searchers,
                                                                                  i,
                                                                                  'uniform',
                                                                                       extreme_value_distribution)
stopCluster(cl)

cl <- makeSOCKcluster(n_clusters)
registerDoSNOW(cl)
treatment_reality_pu_1000 <- foreach(i = 1:n_simulations, 
                                    .combine = rbind,
                                    .multicombine= TRUE,
                                    .packages = c('dplyr', 'Matrix'),                                     
                                    .maxcombine = 2) %dopar% simulate_ind_randomization(airbnb_listings, 
                                                                                  1,
                                                                                  'price',
                                                                                  .75,
                                                                                  sim_info,
                                                                                  max_n_searchers,
                                                                                  i,
                                                                                  'uniform',
                                                                                       extreme_value_distribution)
stopCluster(cl)


cl <- makeSOCKcluster(n_clusters)
registerDoSNOW(cl)
control_reality_pu_250 <- foreach(i = 1:n_simulations, 
                                    .combine = rbind,
                                    .multicombine= TRUE,
                                    .packages = c('dplyr', 'Matrix'),                                  
                                    .maxcombine = 2) %dopar% simulate_ind_randomization(airbnb_listings, 
                                                                                  0,
                                                                                  'price',
                                                                                  .75,
                                                                                  sim_info,
                                                                                  .25*max_n_searchers,
                                                                                  i,
                                                                                  'uniform',
                                                                                       extreme_value_distribution)
stopCluster(cl)

cl <- makeSOCKcluster(n_clusters)
registerDoSNOW(cl)
control_reality_pu_500 <- foreach(i = 1:n_simulations, 
                                    .combine = rbind,
                                    .multicombine= TRUE,
                                    .packages = c('dplyr', 'Matrix'),                                  
                                    .maxcombine = 2) %dopar% simulate_ind_randomization(airbnb_listings, 
                                                                                  0,
                                                                                  'price',
                                                                                  .75,
                                                                                  sim_info,
                                                                                  .5*max_n_searchers,
                                                                                  i,
                                                                                  'uniform',
                                                                                       extreme_value_distribution)
stopCluster(cl)

cl <- makeSOCKcluster(n_clusters)
registerDoSNOW(cl)
control_reality_pu_1000 <- foreach(i = 1:n_simulations, 
                                    .combine = rbind,
                                    .multicombine= TRUE,
                                    .packages = c('dplyr', 'Matrix'),                                   
                                    .maxcombine = 2) %dopar% simulate_ind_randomization(airbnb_listings, 
                                                                                  0,
                                                                                  'price',
                                                                                  .75,
                                                                                  sim_info,
                                                                                  max_n_searchers,
                                                                                  i,
                                                                                  'uniform',
                                                                                       extreme_value_distribution)
stopCluster(cl)

In [ ]:
#Establish baselines
cl <- makeSOCKcluster(n_clusters)
registerDoSNOW(cl)
treatment_reality_uu_250 <- foreach(i = 1:n_simulations, 
                                    .combine = rbind,
                                    .multicombine= TRUE,
                                    .packages = c('dplyr', 'Matrix'),
                                    .maxcombine = 2) %dopar% simulate_ind_randomization(airbnb_listings, 
                                                                                  1,
                                                                                  'unobserved',
                                                                                  .75,
                                                                                  sim_info,
                                                                                  .25*max_n_searchers,
                                                                                  i,
                                                                                  'uniform',
                                                                                        extreme_value_distribution)
stopCluster(cl)

cl <- makeSOCKcluster(n_clusters)
registerDoSNOW(cl)
treatment_reality_uu_500 <- foreach(i = 1:n_simulations, 
                                    .combine = rbind,
                                    .multicombine= TRUE,
                                    .packages = c('dplyr', 'Matrix'),                                    
                                    .maxcombine = 2) %dopar% simulate_ind_randomization(airbnb_listings, 
                                                                                  1,
                                                                                  'unobserved',
                                                                                  .75,
                                                                                  sim_info,
                                                                                  .5*max_n_searchers,
                                                                                  i,
                                                                                  'uniform',
                                                                                       extreme_value_distribution)
stopCluster(cl)

cl <- makeSOCKcluster(n_clusters)
registerDoSNOW(cl)
treatment_reality_uu_1000 <- foreach(i = 1:n_simulations, 
                                    .combine = rbind,
                                    .multicombine= TRUE,
                                    .packages = c('dplyr', 'Matrix'),                                     
                                    .maxcombine = 2) %dopar% simulate_ind_randomization(airbnb_listings, 
                                                                                  1,
                                                                                  'unobserved',
                                                                                  .75,
                                                                                  sim_info,
                                                                                  max_n_searchers,
                                                                                  i,
                                                                                  'uniform',
                                                                                       extreme_value_distribution)
stopCluster(cl)


cl <- makeSOCKcluster(n_clusters)
registerDoSNOW(cl)
control_reality_uu_250 <- foreach(i = 1:n_simulations, 
                                    .combine = rbind,
                                    .multicombine= TRUE,
                                    .packages = c('dplyr', 'Matrix'),                                  
                                    .maxcombine = 2) %dopar% simulate_ind_randomization(airbnb_listings, 
                                                                                  0,
                                                                                  'unobserved',
                                                                                  .75,
                                                                                  sim_info,
                                                                                  .25*max_n_searchers,
                                                                                  i,
                                                                                  'uniform',
                                                                                       extreme_value_distribution)
stopCluster(cl)

cl <- makeSOCKcluster(n_clusters)
registerDoSNOW(cl)
control_reality_uu_500 <- foreach(i = 1:n_simulations, 
                                    .combine = rbind,
                                    .multicombine= TRUE,
                                    .packages = c('dplyr', 'Matrix'),                                  
                                    .maxcombine = 2) %dopar% simulate_ind_randomization(airbnb_listings, 
                                                                                  0,
                                                                                  'unobserved',
                                                                                  .75,
                                                                                  sim_info,
                                                                                  .5*max_n_searchers,
                                                                                  i,
                                                                                  'uniform',
                                                                                       extreme_value_distribution)
stopCluster(cl)

cl <- makeSOCKcluster(n_clusters)
registerDoSNOW(cl)
control_reality_uu_1000 <- foreach(i = 1:n_simulations, 
                                    .combine = rbind,
                                    .multicombine= TRUE,
                                    .packages = c('dplyr', 'Matrix'),                                   
                                    .maxcombine = 2) %dopar% simulate_ind_randomization(airbnb_listings, 
                                                                                  0,
                                                                                  'unobserved',
                                                                                  .75,
                                                                                  sim_info,
                                                                                  max_n_searchers,
                                                                                  i,
                                                                                  'uniform',
                                                                                       extreme_value_distribution)
stopCluster(cl)

In [ ]:
save(control_reality_uu_1000, control_reality_uu_500, control_reality_uu_250,
     control_reality_pu_1000, control_reality_pu_500, control_reality_pu_250,
     control_reality_pn_1000, control_reality_pn_500, control_reality_pn_250,
     control_reality_un_1000, control_reality_un_500, control_reality_un_250,
     treatment_reality_uu_1000, treatment_reality_uu_500, treatment_reality_uu_250,
     treatment_reality_pu_1000, treatment_reality_pu_500, treatment_reality_pu_250,
     treatment_reality_pn_1000, treatment_reality_pn_500, treatment_reality_pn_250,
     treatment_reality_un_1000, treatment_reality_un_500, treatment_reality_un_250,
    file=paste0(
         c(data_directory, 'baseline_results.Rdata'),
         sep='', collapse=''
     ))

In [ ]:
#Establish baselines
cl <- makeSOCKcluster(n_clusters)
registerDoSNOW(cl)
bernoulli_un_250 <- foreach(i = 1:n_simulations, 
                                    .combine = rbind,
                                    .multicombine= TRUE,
                                    .packages = c('dplyr', 'Matrix'),
                                    .maxcombine = 2) %dopar% simulate_ind_randomization(airbnb_listings, 
                                                                                  .5,
                                                                                  'unobserved',
                                                                                  .75,
                                                                                  sim_info,
                                                                                  .25*max_n_searchers,
                                                                                  i,
                                                                                  'normal',
                                                                                        extreme_value_distribution)
stopCluster(cl)

cl <- makeSOCKcluster(n_clusters)
registerDoSNOW(cl)
bernoulli_un_500 <- foreach(i = 1:n_simulations, 
                                    .combine = rbind,
                                    .multicombine= TRUE,
                                    .packages = c('dplyr', 'Matrix'),                                    
                                    .maxcombine = 2) %dopar% simulate_ind_randomization(airbnb_listings, 
                                                                                  .5,
                                                                                  'unobserved',
                                                                                  .75,
                                                                                  sim_info,
                                                                                  .5*max_n_searchers,
                                                                                  i,
                                                                                  'normal',
                                                                                       extreme_value_distribution)
stopCluster(cl)

cl <- makeSOCKcluster(n_clusters)
registerDoSNOW(cl)
bernoulli_un_1000 <- foreach(i = 1:n_simulations, 
                                    .combine = rbind,
                                    .multicombine= TRUE,
                                    .packages = c('dplyr', 'Matrix'),                                     
                                    .maxcombine = 2) %dopar% simulate_ind_randomization(airbnb_listings, 
                                                                                  .5,
                                                                                  'unobserved',
                                                                                  .75,
                                                                                  sim_info,
                                                                                  max_n_searchers,
                                                                                  i,
                                                                                  'normal',
                                                                                       extreme_value_distribution)
stopCluster(cl)

#Establish baselines
cl <- makeSOCKcluster(n_clusters)
registerDoSNOW(cl)
bernoulli_pn_250 <- foreach(i = 1:n_simulations, 
                                    .combine = rbind,
                                    .multicombine= TRUE,
                                    .packages = c('dplyr', 'Matrix'),
                                    .maxcombine = 2) %dopar% simulate_ind_randomization(airbnb_listings, 
                                                                                  .5,
                                                                                  'price',
                                                                                  .75,
                                                                                  sim_info,
                                                                                  .25*max_n_searchers,
                                                                                  i,
                                                                                  'normal',
                                                                                        extreme_value_distribution)
stopCluster(cl)

cl <- makeSOCKcluster(n_clusters)
registerDoSNOW(cl)
bernoulli_pn_500 <- foreach(i = 1:n_simulations, 
                                    .combine = rbind,
                                    .multicombine= TRUE,
                                    .packages = c('dplyr', 'Matrix'),                                    
                                    .maxcombine = 2) %dopar% simulate_ind_randomization(airbnb_listings, 
                                                                                  .5,
                                                                                  'price',
                                                                                  .75,
                                                                                  sim_info,
                                                                                  .5*max_n_searchers,
                                                                                  i,
                                                                                  'normal',
                                                                                       extreme_value_distribution)
stopCluster(cl)

cl <- makeSOCKcluster(n_clusters)
registerDoSNOW(cl)
bernoulli_pn_1000 <- foreach(i = 1:n_simulations, 
                                    .combine = rbind,
                                    .multicombine= TRUE,
                                    .packages = c('dplyr', 'Matrix'),                                     
                                    .maxcombine = 2) %dopar% simulate_ind_randomization(airbnb_listings, 
                                                                                  .5,
                                                                                  'price',
                                                                                  .75,
                                                                                  sim_info,
                                                                                  max_n_searchers,
                                                                                  i,
                                                                                  'normal',
                                                                                       extreme_value_distribution)
stopCluster(cl)

#Establish baselines
cl <- makeSOCKcluster(n_clusters)
registerDoSNOW(cl)
bernoulli_pu_250 <- foreach(i = 1:n_simulations, 
                                    .combine = rbind,
                                    .multicombine= TRUE,
                                    .packages = c('dplyr', 'Matrix'),
                                    .maxcombine = 2) %dopar% simulate_ind_randomization(airbnb_listings, 
                                                                                  .5,
                                                                                  'price',
                                                                                  .75,
                                                                                  sim_info,
                                                                                  .25*max_n_searchers,
                                                                                  i,
                                                                                  'uniform',
                                                                                        extreme_value_distribution)
stopCluster(cl)

cl <- makeSOCKcluster(n_clusters)
registerDoSNOW(cl)
bernoulli_pu_500 <- foreach(i = 1:n_simulations, 
                                    .combine = rbind,
                                    .multicombine= TRUE,
                                    .packages = c('dplyr', 'Matrix'),                                    
                                    .maxcombine = 2) %dopar% simulate_ind_randomization(airbnb_listings, 
                                                                                  .5,
                                                                                  'price',
                                                                                  .75,
                                                                                  sim_info,
                                                                                  .5*max_n_searchers,
                                                                                  i,
                                                                                  'uniform',
                                                                                       extreme_value_distribution)
stopCluster(cl)

cl <- makeSOCKcluster(n_clusters)
registerDoSNOW(cl)
bernoulli_pu_1000 <- foreach(i = 1:n_simulations, 
                                    .combine = rbind,
                                    .multicombine= TRUE,
                                    .packages = c('dplyr', 'Matrix'),                                     
                                    .maxcombine = 2) %dopar% simulate_ind_randomization(airbnb_listings, 
                                                                                  .5,
                                                                                  'price',
                                                                                  .75,
                                                                                  sim_info,
                                                                                  max_n_searchers,
                                                                                  i,
                                                                                  'uniform',
                                                                                       extreme_value_distribution)
stopCluster(cl)

#Establish baselines
cl <- makeSOCKcluster(n_clusters)
registerDoSNOW(cl)
bernoulli_uu_250 <- foreach(i = 1:n_simulations, 
                                    .combine = rbind,
                                    .multicombine= TRUE,
                                    .packages = c('dplyr', 'Matrix'),
                                    .maxcombine = 2) %dopar% simulate_ind_randomization(airbnb_listings, 
                                                                                  .5,
                                                                                  'unobserved',
                                                                                  .75,
                                                                                  sim_info,
                                                                                  .25*max_n_searchers,
                                                                                  i,
                                                                                  'uniform',
                                                                                        extreme_value_distribution)
stopCluster(cl)

cl <- makeSOCKcluster(n_clusters)
registerDoSNOW(cl)
bernoulli_uu_500 <- foreach(i = 1:n_simulations, 
                                    .combine = rbind,
                                    .multicombine= TRUE,
                                    .packages = c('dplyr', 'Matrix'),                                    
                                    .maxcombine = 2) %dopar% simulate_ind_randomization(airbnb_listings, 
                                                                                  .5,
                                                                                  'unobserved',
                                                                                  .75,
                                                                                  sim_info,
                                                                                  .5*max_n_searchers,
                                                                                  i,
                                                                                  'uniform',
                                                                                       extreme_value_distribution)
stopCluster(cl)

cl <- makeSOCKcluster(n_clusters)
registerDoSNOW(cl)
bernoulli_uu_1000 <- foreach(i = 1:n_simulations, 
                                    .combine = rbind,
                                    .multicombine= TRUE,
                                    .packages = c('dplyr', 'Matrix'),                                     
                                    .maxcombine = 2) %dopar% simulate_ind_randomization(airbnb_listings, 
                                                                                  .5,
                                                                                  'unobserved',
                                                                                  .75,
                                                                                  sim_info,
                                                                                  max_n_searchers,
                                                                                  i,
                                                                                  'uniform',
                                                                                       extreme_value_distribution)
stopCluster(cl)

In [ ]:
save(bernoulli_uu_1000, bernoulli_uu_500, bernoulli_uu_250,
    bernoulli_pu_1000, bernoulli_pu_500, bernoulli_pu_250,
    bernoulli_pn_1000, bernoulli_pn_500, bernoulli_pn_250,
    bernoulli_un_1000, bernoulli_un_500, bernoulli_un_250,
    file = paste0(
         c(data_directory, 'bernoulli_results.Rdata'),
         sep='', collapse=''
     ))

In [ ]:
n_clusters <- 18
load(paste0(
         c(data_directory, 'hajek_probabilities_blocked_base.Rdata'),
         sep='', collapse=''
     ))

In [ ]:
set.seed(1204981)
cl <- makeSOCKcluster(n_clusters)
registerDoSNOW(cl)
graph_randomized_pu <- foreach(i = 1:n_simulations, 
                                    .combine = rbind,
                                    .multicombine= TRUE,
                                    .packages = c('dplyr', 'Matrix', 'blockTools', 'lfe', 'igraph'),                                    
                                    .maxcombine = 2) %dopar% simulate_graph_randomization(airbnb_listings, 
                                                                                          .5, 'price', .75, 
                                                                                          sim_info,
                                                                                          max_n_searchers, i, 
                                                                                          'uniform', 
                                                                                          extreme_value_distribution,
                                                                                          clusters_distance_room_type_accom,
                                                                                          graph_distance_room_type_accom,
                                                                                          blocks_base,
                                                                                          c('prob_treat_50_thresh_base', 'prob_control_50_thresh_base',
                                                                                            'prob_treat_75_thresh_base', 'prob_control_75_thresh_base',
                                                                                            'prob_treat_95_thresh_base', 'prob_control_95_thresh_base'),
                                                                                          list(hajek_probabilities_50_50_drta_base, hajek_probabilities_75_50_drta_base,
                                                                                               hajek_probabilities_95_50_drta_base))
stopCluster(cl)

In [ ]:
set.seed(12041248)
cl <- makeSOCKcluster(n_clusters)
registerDoSNOW(cl)
graph_randomized_pn <- foreach(i = 1:n_simulations, 
                                    .combine = rbind,
                                    .multicombine= TRUE,
                                    .packages = c('dplyr', 'Matrix', 'blockTools', 'lfe', 'igraph'),                                    
                                    .maxcombine = 2) %dopar% simulate_graph_randomization(airbnb_listings, 
                                                                                          .5, 'price', .75, 
                                                                                          sim_info,
                                                                                          max_n_searchers, i, 
                                                                                          'normal', 
                                                                                          extreme_value_distribution,
                                                                                          clusters_distance_room_type_accom,
                                                                                          graph_distance_room_type_accom,
                                                                                          blocks_base,
                                                                                          c('prob_treat_50_thresh_base', 'prob_control_50_thresh_base',
                                                                                            'prob_treat_75_thresh_base', 'prob_control_75_thresh_base',
                                                                                            'prob_treat_95_thresh_base', 'prob_control_95_thresh_base'),
                                                                                          list(hajek_probabilities_50_50_drta_base, hajek_probabilities_75_50_drta_base,
                                                                                               hajek_probabilities_95_50_drta_base))
stopCluster(cl)

In [ ]:
set.seed(12049010)
cl <- makeSOCKcluster(n_clusters)
registerDoSNOW(cl)
graph_randomized_un <- foreach(i = 1:n_simulations, 
                                    .combine = rbind,
                                    .multicombine= TRUE,
                                    .packages = c('dplyr', 'Matrix', 'blockTools', 'lfe', 'igraph'),                                    
                                    .maxcombine = 2) %dopar% simulate_graph_randomization(airbnb_listings, 
                                                                                          .5, 'unobserved', .75, 
                                                                                          sim_info,
                                                                                          max_n_searchers, i, 
                                                                                          'normal', 
                                                                                          extreme_value_distribution,
                                                                                          clusters_distance_room_type_accom,
                                                                                          graph_distance_room_type_accom,
                                                                                          blocks_base,
                                                                                          c('prob_treat_50_thresh_base', 'prob_control_50_thresh_base',
                                                                                            'prob_treat_75_thresh_base', 'prob_control_75_thresh_base',
                                                                                            'prob_treat_95_thresh_base', 'prob_control_95_thresh_base'),
                                                                                          list(hajek_probabilities_50_50_drta_base, hajek_probabilities_75_50_drta_base,
                                                                                               hajek_probabilities_95_50_drta_base))
stopCluster(cl)

In [ ]:
set.seed(120497010)
cl <- makeSOCKcluster(n_clusters)
registerDoSNOW(cl)
graph_randomized_uu <- foreach(i = 1:n_simulations, 
                                    .combine = rbind,
                                    .multicombine= TRUE,
                                    .packages = c('dplyr', 'Matrix', 'blockTools', 'lfe', 'igraph'),                                    
                                    .maxcombine = 2) %dopar% simulate_graph_randomization(airbnb_listings, 
                                                                                          .5, 'unobserved', .75, 
                                                                                          sim_info,
                                                                                          max_n_searchers, i, 
                                                                                          'uniform', 
                                                                                          extreme_value_distribution,
                                                                                          clusters_distance_room_type_accom,
                                                                                          graph_distance_room_type_accom,
                                                                                          blocks_base,
                                                                                          c('prob_treat_50_thresh_base', 'prob_control_50_thresh_base',
                                                                                            'prob_treat_75_thresh_base', 'prob_control_75_thresh_base',
                                                                                            'prob_treat_95_thresh_base', 'prob_control_95_thresh_base'),
                                                                                          list(hajek_probabilities_50_50_drta_base, hajek_probabilities_75_50_drta_base,
                                                                                               hajek_probabilities_95_50_drta_base))
stopCluster(cl)

In [ ]:
save(graph_randomized_pu, graph_randomized_pn, 
    graph_randomized_un, graph_randomized_uu,
    file = paste0(
         c(data_directory, 'graph_randomized_results.Rdata'),
         sep='', collapse=''
     ))

In [ ]:
rm(hajek_probabilities_50_50_drta_base, hajek_probabilities_75_50_drta_base, hajek_probabilities_95_50_drta_base)

In [ ]:
n_clusters <- 18
load(paste0(
         c(data_directory, 'hajek_probabilities_blocked_base_100.Rdata'),
         sep='', collapse=''
     ))

In [ ]:
set.seed(2409810)
cl <- makeSOCKcluster(n_clusters)
registerDoSNOW(cl)
graph_randomized_pn_100 <- foreach(i = 1:n_simulations, 
                                    .combine = rbind,
                                    .multicombine= TRUE,
                                    .packages = c('dplyr', 'Matrix', 'blockTools', 'lfe', 'igraph'),                                    
                                    .maxcombine = 2) %dopar% simulate_graph_randomization(airbnb_listings, 
                                                                                          .5, 'price', .75, 
                                                                                          sim_info,
                                                                                          max_n_searchers, i, 
                                                                                          'normal', 
                                                                                          extreme_value_distribution,
                                                                                          clusters_distance_room_type_accom_100,
                                                                                          graph_distance_room_type_accom,
                                                                                          blocks_base_100,
                                                                                          c('prob_treat_50_thresh_base_100', 'prob_control_50_thresh_base_100',
                                                                                            'prob_treat_75_thresh_base_100', 'prob_control_75_thresh_base_100',
                                                                                            'prob_treat_95_thresh_base_100', 'prob_control_95_thresh_base_100'),
                                                                                          list(hajek_probabilities_50_50_drta_base_100, hajek_probabilities_75_50_drta_base_100,
                                                                                               hajek_probabilities_95_50_drta_base_100))
stopCluster(cl)

In [ ]:
set.seed(12409801)
cl <- makeSOCKcluster(n_clusters)
registerDoSNOW(cl)
graph_randomized_un_100 <- foreach(i = 1:n_simulations, 
                                    .combine = rbind,
                                    .multicombine= TRUE,
                                    .packages = c('dplyr', 'Matrix', 'blockTools', 'lfe', 'igraph'),                                    
                                    .maxcombine = 2) %dopar% simulate_graph_randomization(airbnb_listings, 
                                                                                          .5, 'unobserved', .75, 
                                                                                          sim_info,
                                                                                          max_n_searchers, i, 
                                                                                          'normal', 
                                                                                          extreme_value_distribution,
                                                                                          clusters_distance_room_type_accom_100,
                                                                                          graph_distance_room_type_accom,
                                                                                          blocks_base_100,
                                                                                          c('prob_treat_50_thresh_base_100', 'prob_control_50_thresh_base_100',
                                                                                            'prob_treat_75_thresh_base_100', 'prob_control_75_thresh_base_100',
                                                                                            'prob_treat_95_thresh_base_100', 'prob_control_95_thresh_base_100'),
                                                                                          list(hajek_probabilities_50_50_drta_base_100, hajek_probabilities_75_50_drta_base_100,
                                                                                               hajek_probabilities_95_50_drta_base_100))
stopCluster(cl)

In [ ]:
save(graph_randomized_pn_100, graph_randomized_un_100, 
    file = paste0(
         c(data_directory, 'graph_randomized_results_100.Rdata'),
         sep='', collapse=''
     ))

In [ ]:
rm(hajek_probabilities_50_50_drta_base_100, hajek_probabilities_75_50_drta_base_100, hajek_probabilities_95_50_drta_base_100)

In [ ]:
n_clusters <- 20
load(paste0(
         c(data_directory, 'hajek_probabilities_blocked_base_200.Rdata'),
         sep='', collapse=''
     ))

In [ ]:
set.seed(3049810)
cl <- makeSOCKcluster(n_clusters)
registerDoSNOW(cl)
graph_randomized_pn_200 <- foreach(i = 1:n_simulations, 
                                    .combine = rbind,
                                    .multicombine= TRUE,
                                    .packages = c('dplyr', 'Matrix', 'blockTools', 'lfe', 'igraph'),                                    
                                    .maxcombine = 2) %dopar% simulate_graph_randomization(airbnb_listings, 
                                                                                          .5, 'price', .75, 
                                                                                          sim_info,
                                                                                          max_n_searchers, i, 
                                                                                          'normal', 
                                                                                          extreme_value_distribution,
                                                                                          clusters_distance_room_type_accom_200,
                                                                                          graph_distance_room_type_accom,
                                                                                          blocks_base_200,
                                                                                          c('prob_treat_50_thresh_base_200', 'prob_control_50_thresh_base_200',
                                                                                            'prob_treat_75_thresh_base_200', 'prob_control_75_thresh_base_200',
                                                                                            'prob_treat_95_thresh_base_200', 'prob_control_95_thresh_base_200'),
                                                                                          list(hajek_probabilities_50_50_drta_base_200, hajek_probabilities_75_50_drta_base_200,
                                                                                               hajek_probabilities_95_50_drta_base_200))
stopCluster(cl)

In [ ]:
set.seed(32405801)
cl <- makeSOCKcluster(n_clusters)
registerDoSNOW(cl)
graph_randomized_un_200 <- foreach(i = 1:n_simulations, 
                                    .combine = rbind,
                                    .multicombine= TRUE,
                                    .packages = c('dplyr', 'Matrix', 'blockTools', 'lfe', 'igraph'),                                    
                                    .maxcombine = 2) %dopar% simulate_graph_randomization(airbnb_listings, 
                                                                                          .5, 'unobserved', .75, 
                                                                                          sim_info,
                                                                                          max_n_searchers, i, 
                                                                                          'normal', 
                                                                                          extreme_value_distribution,
                                                                                          clusters_distance_room_type_accom_200,
                                                                                          graph_distance_room_type_accom,
                                                                                          blocks_base_200,
                                                                                          c('prob_treat_50_thresh_base_200', 'prob_control_50_thresh_base_200',
                                                                                            'prob_treat_75_thresh_base_200', 'prob_control_75_thresh_base_200',
                                                                                            'prob_treat_95_thresh_base_200', 'prob_control_95_thresh_base_200'),
                                                                                          list(hajek_probabilities_50_50_drta_base_200, hajek_probabilities_75_50_drta_base_200,
                                                                                               hajek_probabilities_95_50_drta_base_200))
stopCluster(cl)

In [ ]:
save(graph_randomized_pn_200, graph_randomized_un_200, 
    file = paste0(
         c(data_directory, 'graph_randomized_results_200.Rdata'),
         sep='', collapse=''
     ))

In [ ]:
rm(hajek_probabilities_50_50_drta_base_200, hajek_probabilities_75_50_drta_base_200, hajek_probabilities_95_50_drta_base_200)

In [ ]:
n_clusters <- 20
load(paste0(
         c(data_directory, 'hajek_probabilities_blocked_base_500.Rdata'),
         sep='', collapse=''
     ))

In [ ]:
set.seed(4549810)
cl <- makeSOCKcluster(n_clusters)
registerDoSNOW(cl)
graph_randomized_pn_500 <- foreach(i = 1:n_simulations, 
                                    .combine = rbind,
                                    .multicombine= TRUE,
                                    .packages = c('dplyr', 'Matrix', 'blockTools', 'lfe', 'igraph'),                                    
                                    .maxcombine = 2) %dopar% simulate_graph_randomization(airbnb_listings, 
                                                                                          .5, 'price', .75, 
                                                                                          sim_info,
                                                                                          max_n_searchers, i, 
                                                                                          'normal', 
                                                                                          extreme_value_distribution,
                                                                                          clusters_distance_room_type_accom_500,
                                                                                          graph_distance_room_type_accom,
                                                                                          blocks_base_500,
                                                                                          c('prob_treat_50_thresh_base_500', 'prob_control_50_thresh_base_500',
                                                                                            'prob_treat_75_thresh_base_500', 'prob_control_75_thresh_base_500',
                                                                                            'prob_treat_95_thresh_base_500', 'prob_control_95_thresh_base_500'),
                                                                                          list(hajek_probabilities_50_50_drta_base_500, hajek_probabilities_75_50_drta_base_500,
                                                                                               hajek_probabilities_95_50_drta_base_500))
stopCluster(cl)

In [ ]:
set.seed(32405801)
cl <- makeSOCKcluster(n_clusters)
registerDoSNOW(cl)
graph_randomized_un_500 <- foreach(i = 1:n_simulations, 
                                    .combine = rbind,
                                    .multicombine= TRUE,
                                    .packages = c('dplyr', 'Matrix', 'blockTools', 'lfe', 'igraph'),                                    
                                    .maxcombine = 2) %dopar% simulate_graph_randomization(airbnb_listings, 
                                                                                          .5, 'unobserved', .75, 
                                                                                          sim_info,
                                                                                          max_n_searchers, i, 
                                                                                          'normal', 
                                                                                          extreme_value_distribution,
                                                                                          clusters_distance_room_type_accom_500,
                                                                                          graph_distance_room_type_accom,
                                                                                          blocks_base_500,
                                                                                          c('prob_treat_50_thresh_base_500', 'prob_control_50_thresh_base_500',
                                                                                            'prob_treat_75_thresh_base_500', 'prob_control_75_thresh_base_500',
                                                                                            'prob_treat_95_thresh_base_500', 'prob_control_95_thresh_base_500'),
                                                                                          list(hajek_probabilities_50_50_drta_base_500, hajek_probabilities_75_50_drta_base_500,
                                                                                               hajek_probabilities_95_50_drta_base_500))
stopCluster(cl)

In [ ]:
save(graph_randomized_pn_500, graph_randomized_un_500, 
    file = paste0(
         c(data_directory, 'graph_randomized_results_500.Rdata'),
         sep='', collapse=''
     ))

In [ ]:
rm(hajek_probabilities_50_50_drta_base_500, hajek_probabilities_75_50_drta_base_500, hajek_probabilities_95_50_drta_base_500)

In [ ]:
n_clusters <- 20
load(paste0(
         c(data_directory, 'hajek_probabilities_blocked_base_1000.Rdata'),
         sep='', collapse=''
     ))

In [ ]:
set.seed(41249810)
cl <- makeSOCKcluster(n_clusters)
registerDoSNOW(cl)
graph_randomized_pn_1000 <- foreach(i = 1:n_simulations, 
                                    .combine = rbind,
                                    .multicombine= TRUE,
                                    .packages = c('dplyr', 'Matrix', 'blockTools', 'lfe', 'igraph'),                                    
                                    .maxcombine = 2) %dopar% simulate_graph_randomization(airbnb_listings, 
                                                                                          .5, 'price', .75, 
                                                                                          sim_info,
                                                                                          max_n_searchers, i, 
                                                                                          'normal', 
                                                                                          extreme_value_distribution,
                                                                                          clusters_distance_room_type_accom_1000,
                                                                                          graph_distance_room_type_accom,
                                                                                          blocks_base_1000,
                                                                                          c('prob_treat_50_thresh_base_1000', 'prob_control_50_thresh_base_1000',
                                                                                            'prob_treat_75_thresh_base_1000', 'prob_control_75_thresh_base_1000',
                                                                                            'prob_treat_95_thresh_base_1000', 'prob_control_95_thresh_base_1000'),
                                                                                          list(hajek_probabilities_50_50_drta_base_1000, hajek_probabilities_75_50_drta_base_1000,
                                                                                               hajek_probabilities_95_50_drta_base_1000))
stopCluster(cl)

In [ ]:
set.seed(12309234)
cl <- makeSOCKcluster(n_clusters)
registerDoSNOW(cl)
graph_randomized_un_1000 <- foreach(i = 1:n_simulations, 
                                    .combine = rbind,
                                    .multicombine= TRUE,
                                    .packages = c('dplyr', 'Matrix', 'blockTools', 'lfe', 'igraph'),                                    
                                    .maxcombine = 2) %dopar% simulate_graph_randomization(airbnb_listings, 
                                                                                          .5, 'unobserved', .75, 
                                                                                          sim_info,
                                                                                          max_n_searchers, i, 
                                                                                          'normal', 
                                                                                          extreme_value_distribution,
                                                                                          clusters_distance_room_type_accom_1000,
                                                                                          graph_distance_room_type_accom,
                                                                                          blocks_base_1000,
                                                                                          c('prob_treat_50_thresh_base_1000', 'prob_control_50_thresh_base_1000',
                                                                                            'prob_treat_75_thresh_base_1000', 'prob_control_75_thresh_base_1000',
                                                                                            'prob_treat_95_thresh_base_1000', 'prob_control_95_thresh_base_1000'),
                                                                                          list(hajek_probabilities_50_50_drta_base_1000, hajek_probabilities_75_50_drta_base_1000,
                                                                                               hajek_probabilities_95_50_drta_base_1000))
stopCluster(cl)

In [ ]:
save(graph_randomized_pn_1000, graph_randomized_un_1000, 
    file = paste0(
         c(data_directory, 'graph_randomized_results_1000.Rdata'),
         sep='', collapse=''
     ))

In [ ]:
rm(hajek_probabilities_50_50_drta_base_1000, hajek_probabilities_75_50_drta_base_1000, hajek_probabilities_95_50_drta_base_1000)

In [ ]:
n_clusters <- 20
load(paste0(
         c(data_directory, 'hajek_probabilities_blocked_base_rw_01.Rdata'),
         sep='', collapse=''
     ))

In [ ]:
set.seed(12409841)
cl <- makeSOCKcluster(n_clusters)
registerDoSNOW(cl)
graph_randomized_pn_rw_01 <- foreach(i = 1:n_simulations, 
                                    .combine = rbind,
                                    .multicombine= TRUE,
                                    .packages = c('dplyr', 'Matrix', 'blockTools', 'lfe', 'igraph'),                                    
                                    .maxcombine = 2) %dopar% simulate_graph_randomization(airbnb_listings, 
                                                                                          .5, 'price', .75, 
                                                                                          sim_info,
                                                                                          max_n_searchers, i, 
                                                                                          'normal', 
                                                                                          extreme_value_distribution,
                                                                                          clusters_distance_room_type_accom_rw_01,
                                                                                          graph_distance_room_type_accom_rw_01,
                                                                                          blocks_base_rw_01,
                                                                                          c('prob_treat_50_thresh_base_rw_01', 'prob_control_50_thresh_base_rw_01',
                                                                                            'prob_treat_75_thresh_base_rw_01', 'prob_control_75_thresh_base_rw_01',
                                                                                            'prob_treat_95_thresh_base_rw_01', 'prob_control_95_thresh_base_rw_01'),
                                                                                          list(hajek_probabilities_50_50_drta_base_rw_01, hajek_probabilities_75_50_drta_base_rw_01,
                                                                                               hajek_probabilities_95_50_drta_base_rw_01))
stopCluster(cl)

In [ ]:
set.seed(712049710)
cl <- makeSOCKcluster(n_clusters)
registerDoSNOW(cl)
graph_randomized_un_rw_01 <- foreach(i = 1:n_simulations, 
                                    .combine = rbind,
                                    .multicombine= TRUE,
                                    .packages = c('dplyr', 'Matrix', 'blockTools', 'lfe', 'igraph'),                                    
                                    .maxcombine = 2) %dopar% simulate_graph_randomization(airbnb_listings, 
                                                                                          .5, 'unobserved', .75, 
                                                                                          sim_info,
                                                                                          max_n_searchers, i, 
                                                                                          'normal', 
                                                                                          extreme_value_distribution,
                                                                                          clusters_distance_room_type_accom_rw_01,
                                                                                          graph_distance_room_type_accom_rw_01,
                                                                                          blocks_base_rw_01,
                                                                                          c('prob_treat_50_thresh_base_rw_01', 'prob_control_50_thresh_base_rw_01',
                                                                                            'prob_treat_75_thresh_base_rw_01', 'prob_control_75_thresh_base_rw_01',
                                                                                            'prob_treat_95_thresh_base_rw_01', 'prob_control_95_thresh_base_rw_01'),
                                                                                          list(hajek_probabilities_50_50_drta_base_rw_01, hajek_probabilities_75_50_drta_base_rw_01,
                                                                                               hajek_probabilities_95_50_drta_base_rw_01))
stopCluster(cl)

In [ ]:
save(graph_randomized_pn_rw_01, graph_randomized_un_rw_01, 
    file = paste0(
         c(data_directory, 'graph_randomized_results_rw_01.Rdata'),
         sep='', collapse=''
     ))

In [ ]:
rm(hajek_probabilities_50_50_drta_base_rw_01, hajek_probabilities_75_50_drta_base_rw_01, hajek_probabilities_95_50_drta_base_rw_01)

In [ ]:
n_clusters <- 20
load(paste0(
         c(data_directory, 'hajek_probabilities_blocked_base_rw_02.Rdata'),
         sep='', collapse=''
     ))

In [ ]:
set.seed(4091284)
cl <- makeSOCKcluster(n_clusters)
registerDoSNOW(cl)
graph_randomized_pn_rw_02 <- foreach(i = 1:n_simulations, 
                                    .combine = rbind,
                                    .multicombine= TRUE,
                                    .packages = c('dplyr', 'Matrix', 'blockTools', 'lfe', 'igraph'),                                    
                                    .maxcombine = 2) %dopar% simulate_graph_randomization(airbnb_listings, 
                                                                                          .5, 'price', .75, 
                                                                                          sim_info,
                                                                                          max_n_searchers, i, 
                                                                                          'normal', 
                                                                                          extreme_value_distribution,
                                                                                          clusters_distance_room_type_accom_rw_02,
                                                                                          graph_distance_room_type_accom_rw_02,
                                                                                          blocks_base_rw_02,
                                                                                          c('prob_treat_50_thresh_base_rw_02', 'prob_control_50_thresh_base_rw_02',
                                                                                            'prob_treat_75_thresh_base_rw_02', 'prob_control_75_thresh_base_rw_02',
                                                                                            'prob_treat_95_thresh_base_rw_02', 'prob_control_95_thresh_base_rw_02'),
                                                                                          list(hajek_probabilities_50_50_drta_base_rw_02, hajek_probabilities_75_50_drta_base_rw_02,
                                                                                               hajek_probabilities_95_50_drta_base_rw_02))
stopCluster(cl)

In [ ]:
set.seed(712049710)
cl <- makeSOCKcluster(n_clusters)
registerDoSNOW(cl)
graph_randomized_un_rw_02 <- foreach(i = 1:n_simulations, 
                                    .combine = rbind,
                                    .multicombine= TRUE,
                                    .packages = c('dplyr', 'Matrix', 'blockTools', 'lfe', 'igraph'),                                    
                                    .maxcombine = 2) %dopar% simulate_graph_randomization(airbnb_listings, 
                                                                                          .5, 'unobserved', .75, 
                                                                                          sim_info,
                                                                                          max_n_searchers, i, 
                                                                                          'normal', 
                                                                                          extreme_value_distribution,
                                                                                          clusters_distance_room_type_accom_rw_02,
                                                                                          graph_distance_room_type_accom_rw_02,
                                                                                          blocks_base_rw_02,
                                                                                          c('prob_treat_50_thresh_base_rw_02', 'prob_control_50_thresh_base_rw_02',
                                                                                            'prob_treat_75_thresh_base_rw_02', 'prob_control_75_thresh_base_rw_02',
                                                                                            'prob_treat_95_thresh_base_rw_02', 'prob_control_95_thresh_base_rw_02'),
                                                                                          list(hajek_probabilities_50_50_drta_base_rw_02, hajek_probabilities_75_50_drta_base_rw_02,
                                                                                               hajek_probabilities_95_50_drta_base_rw_02))
stopCluster(cl)

In [ ]:
save(graph_randomized_pn_rw_02, graph_randomized_un_rw_02, 
    file = paste0(
         c(data_directory, 'graph_randomized_results_rw_02.Rdata'),
         sep='', collapse=''
     ))

In [ ]:
rm(hajek_probabilities_50_50_drta_base_rw_02, hajek_probabilities_75_50_drta_base_rw_02, hajek_probabilities_95_50_drta_base_rw_02)

In [ ]:
n_clusters <- 20
load(paste0(
         c(data_directory, 'hajek_probabilities_blocked_base_rw_05.Rdata'),
         sep='', collapse=''
     ))

In [ ]:
set.seed(76410640)
cl <- makeSOCKcluster(n_clusters)
registerDoSNOW(cl)
graph_randomized_pn_rw_05 <- foreach(i = 1:n_simulations, 
                                    .combine = rbind,
                                    .multicombine= TRUE,
                                    .packages = c('dplyr', 'Matrix', 'blockTools', 'lfe', 'igraph'),                                    
                                    .maxcombine = 2) %dopar% simulate_graph_randomization(airbnb_listings, 
                                                                                          .5, 'price', .75, 
                                                                                          sim_info,
                                                                                          max_n_searchers, i, 
                                                                                          'normal', 
                                                                                          extreme_value_distribution,
                                                                                          clusters_distance_room_type_accom_rw_05,
                                                                                          graph_distance_room_type_accom_rw_05,
                                                                                          blocks_base_rw_05,
                                                                                          c('prob_treat_50_thresh_base_rw_05', 'prob_control_50_thresh_base_rw_05',
                                                                                            'prob_treat_75_thresh_base_rw_05', 'prob_control_75_thresh_base_rw_05',
                                                                                            'prob_treat_95_thresh_base_rw_05', 'prob_control_95_thresh_base_rw_05'),
                                                                                          list(hajek_probabilities_50_50_drta_base_rw_05, hajek_probabilities_75_50_drta_base_rw_05,
                                                                                               hajek_probabilities_95_50_drta_base_rw_05))
stopCluster(cl)

In [ ]:
set.seed(712049710)
cl <- makeSOCKcluster(n_clusters)
registerDoSNOW(cl)
graph_randomized_un_rw_05 <- foreach(i = 1:n_simulations, 
                                    .combine = rbind,
                                    .multicombine= TRUE,
                                    .packages = c('dplyr', 'Matrix', 'blockTools', 'lfe', 'igraph'),                                    
                                    .maxcombine = 2) %dopar% simulate_graph_randomization(airbnb_listings, 
                                                                                          .5, 'unobserved', .75, 
                                                                                          sim_info,
                                                                                          max_n_searchers, i, 
                                                                                          'normal', 
                                                                                          extreme_value_distribution,
                                                                                          clusters_distance_room_type_accom_rw_05,
                                                                                          graph_distance_room_type_accom_rw_05,
                                                                                          blocks_base_rw_05,
                                                                                          c('prob_treat_50_thresh_base_rw_05', 'prob_control_50_thresh_base_rw_05',
                                                                                            'prob_treat_75_thresh_base_rw_05', 'prob_control_75_thresh_base_rw_05',
                                                                                            'prob_treat_95_thresh_base_rw_05', 'prob_control_95_thresh_base_rw_05'),
                                                                                          list(hajek_probabilities_50_50_drta_base_rw_05, hajek_probabilities_75_50_drta_base_rw_05,
                                                                                               hajek_probabilities_95_50_drta_base_rw_05))
stopCluster(cl)

In [ ]:
save(graph_randomized_pn_rw_05, graph_randomized_un_rw_05, 
    file = paste0(
         c(data_directory, 'graph_randomized_results_rw_05.Rdata'),
         sep='', collapse=''
     ))

In [ ]:
rm(hajek_probabilities_50_50_drta_base_rw_05, hajek_probabilities_75_50_drta_base_rw_05, hajek_probabilities_95_50_drta_base_rw_05)

In [ ]:
n_clusters <- 20
load(paste0(
         c(data_directory, 'hajek_probabilities_blocked_base_rw_10.Rdata'),
         sep='', collapse=''
     ))

In [ ]:
set.seed(25812470)
cl <- makeSOCKcluster(n_clusters, outfile=paste0(c(data_directory, 'rw_10_pn.txt'), sep='', collapse=''))
registerDoSNOW(cl)
graph_randomized_pn_rw_10 <- foreach(i = 1:n_simulations, 
                                    .combine = rbind,
                                    .multicombine= TRUE,
                                    .packages = c('dplyr', 'Matrix', 'blockTools', 'lfe', 'igraph'),                                    
                                    .maxcombine = 2) %dopar% simulate_graph_randomization(airbnb_listings, 
                                                                                          .5, 'price', .75, 
                                                                                          sim_info,
                                                                                          max_n_searchers, i, 
                                                                                          'normal', 
                                                                                          extreme_value_distribution,
                                                                                          clusters_distance_room_type_accom_rw_10,
                                                                                          graph_distance_room_type_accom_rw_10,
                                                                                          blocks_base_rw_10,
                                                                                          c('prob_treat_50_thresh_base_rw_10', 'prob_control_50_thresh_base_rw_10',
                                                                                            'prob_treat_75_thresh_base_rw_10', 'prob_control_75_thresh_base_rw_10',
                                                                                            'prob_treat_95_thresh_base_rw_10', 'prob_control_95_thresh_base_rw_10'),
                                                                                          list(hajek_probabilities_50_50_drta_base_rw_10, hajek_probabilities_75_50_drta_base_rw_10,
                                                                                               hajek_probabilities_95_50_drta_base_rw_10))
stopCluster(cl)

In [ ]:
set.seed(932049710)
cl <- makeSOCKcluster(n_clusters)
registerDoSNOW(cl)
graph_randomized_un_rw_10 <- foreach(i = 1:n_simulations, 
                                    .combine = rbind,
                                    .multicombine= TRUE,
                                    .packages = c('dplyr', 'Matrix', 'blockTools', 'lfe', 'igraph'),                                    
                                    .maxcombine = 2) %dopar% simulate_graph_randomization(airbnb_listings, 
                                                                                          .5, 'unobserved', .75, 
                                                                                          sim_info,
                                                                                          max_n_searchers, i, 
                                                                                          'normal', 
                                                                                          extreme_value_distribution,
                                                                                          clusters_distance_room_type_accom_rw_10,
                                                                                          graph_distance_room_type_accom_rw_10,
                                                                                          blocks_base_rw_10,
                                                                                          c('prob_treat_50_thresh_base_rw_10', 'prob_control_50_thresh_base_rw_10',
                                                                                            'prob_treat_75_thresh_base_rw_10', 'prob_control_75_thresh_base_rw_10',
                                                                                            'prob_treat_95_thresh_base_rw_10', 'prob_control_95_thresh_base_rw_10'),
                                                                                          list(hajek_probabilities_50_50_drta_base_rw_10, hajek_probabilities_75_50_drta_base_rw_10,
                                                                                               hajek_probabilities_95_50_drta_base_rw_10))
stopCluster(cl)

In [ ]:
save(graph_randomized_pn_rw_10, graph_randomized_un_rw_10, 
    file = paste0(
         c(data_directory, 'graph_randomized_results_rw_10.Rdata'),
         sep='', collapse=''
     ))

In [ ]:
rm(hajek_probabilities_50_50_drta_base_rw_10, hajek_probabilities_75_50_drta_base_rw_10, hajek_probabilities_95_50_drta_base_rw_10)

In [ ]:
n_clusters <- 20
load(paste0(
         c(data_directory, 'hajek_probabilities_blocked_base_rw_15.Rdata'),
         sep='', collapse=''
     ))

In [ ]:
set.seed(462470)
cl <- makeSOCKcluster(n_clusters)
registerDoSNOW(cl)
graph_randomized_pn_rw_15 <- foreach(i = 1:n_simulations, 
                                    .combine = rbind,
                                    .multicombine= TRUE,
                                    .packages = c('dplyr', 'Matrix', 'blockTools', 'lfe', 'igraph'),                                    
                                    .maxcombine = 2) %dopar% simulate_graph_randomization(airbnb_listings, 
                                                                                          .5, 'price', .75, 
                                                                                          sim_info,
                                                                                          max_n_searchers, i, 
                                                                                          'normal', 
                                                                                          extreme_value_distribution,
                                                                                          clusters_distance_room_type_accom_rw_15,
                                                                                          graph_distance_room_type_accom_rw_15,
                                                                                          blocks_base_rw_15,
                                                                                          c('prob_treat_50_thresh_base_rw_15', 'prob_control_50_thresh_base_rw_15',
                                                                                            'prob_treat_75_thresh_base_rw_15', 'prob_control_75_thresh_base_rw_15',
                                                                                            'prob_treat_95_thresh_base_rw_15', 'prob_control_95_thresh_base_rw_15'),
                                                                                          list(hajek_probabilities_50_50_drta_base_rw_15, hajek_probabilities_75_50_drta_base_rw_15,
                                                                                               hajek_probabilities_95_50_drta_base_rw_15))
stopCluster(cl)

In [ ]:
set.seed(572049710)
cl <- makeSOCKcluster(n_clusters)
registerDoSNOW(cl)
graph_randomized_un_rw_15 <- foreach(i = 1:n_simulations, 
                                    .combine = rbind,
                                    .multicombine= TRUE,
                                    .packages = c('dplyr', 'Matrix', 'blockTools', 'lfe', 'igraph'),                                    
                                    .maxcombine = 2) %dopar% simulate_graph_randomization(airbnb_listings, 
                                                                                          .5, 'unobserved', .75, 
                                                                                          sim_info,
                                                                                          max_n_searchers, i, 
                                                                                          'normal', 
                                                                                          extreme_value_distribution,
                                                                                          clusters_distance_room_type_accom_rw_15,
                                                                                          graph_distance_room_type_accom_rw_15,
                                                                                          blocks_base_rw_15,
                                                                                          c('prob_treat_50_thresh_base_rw_15', 'prob_control_50_thresh_base_rw_15',
                                                                                            'prob_treat_75_thresh_base_rw_15', 'prob_control_75_thresh_base_rw_15',
                                                                                            'prob_treat_95_thresh_base_rw_15', 'prob_control_95_thresh_base_rw_15'),
                                                                                          list(hajek_probabilities_50_50_drta_base_rw_15, hajek_probabilities_75_50_drta_base_rw_15,
                                                                                               hajek_probabilities_95_50_drta_base_rw_15))
stopCluster(cl)

In [ ]:
save(graph_randomized_pn_rw_15, graph_randomized_un_rw_15, 
    file = paste0(
         c(data_directory, 'graph_randomized_results_rw_15'),
         sep='', collapse=''
     ))

In [ ]:
rm(hajek_probabilities_50_50_drta_base_rw_15, hajek_probabilities_75_50_drta_base_rw_15, hajek_probabilities_95_50_drta_base_rw_15)

In [ ]:
n_clusters <- 20
load(paste0(
         c(data_directory, 'hajek_probabilities_blocked_base.Rdata'),
         sep='', collapse=''
     ))

In [ ]:
set.seed(47641248)
cl <- makeSOCKcluster(n_clusters, outfile=paste0(c(data_directory, 'pn_u250_logs.txt'), sep='', collapse=''))
registerDoSNOW(cl)
graph_randomized_pn_u250 <- foreach(i = 1:n_simulations, 
                                    .combine = rbind,
                                    .multicombine= TRUE,
                                    .packages = c('dplyr', 'Matrix', 'blockTools', 'lfe', 'igraph'),                                    
                                    .maxcombine = 2) %dopar% simulate_graph_randomization(airbnb_listings, 
                                                                                          .5, 'price', .75, 
                                                                                          sim_info,
                                                                                          .25*max_n_searchers, i, 
                                                                                          'normal', 
                                                                                          extreme_value_distribution,
                                                                                          clusters_distance_room_type_accom,
                                                                                          graph_distance_room_type_accom,
                                                                                          blocks_base,
                                                                                          c('prob_treat_50_thresh_base', 'prob_control_50_thresh_base',
                                                                                            'prob_treat_75_thresh_base', 'prob_control_75_thresh_base',
                                                                                            'prob_treat_95_thresh_base', 'prob_control_95_thresh_base'),
                                                                                          list(hajek_probabilities_50_50_drta_base, hajek_probabilities_75_50_drta_base,
                                                                                               hajek_probabilities_95_50_drta_base))
stopCluster(cl)

cl <- makeSOCKcluster(n_clusters, outfile=paste0(c(data_directory, 'pn_u500_logs.txt'), sep='', collapse=''))
registerDoSNOW(cl)
graph_randomized_pn_u500 <- foreach(i = 1:n_simulations, 
                                    .combine = rbind,
                                    .multicombine= TRUE,
                                    .packages = c('dplyr', 'Matrix', 'blockTools', 'lfe', 'igraph'),                                    
                                    .maxcombine = 2) %dopar% simulate_graph_randomization(airbnb_listings, 
                                                                                          .5, 'price', .75, 
                                                                                          sim_info,
                                                                                          .5*max_n_searchers, i, 
                                                                                          'normal', 
                                                                                          extreme_value_distribution,
                                                                                          clusters_distance_room_type_accom,
                                                                                          graph_distance_room_type_accom,
                                                                                          blocks_base,
                                                                                          c('prob_treat_50_thresh_base', 'prob_control_50_thresh_base',
                                                                                            'prob_treat_75_thresh_base', 'prob_control_75_thresh_base',
                                                                                            'prob_treat_95_thresh_base', 'prob_control_95_thresh_base'),
                                                                                          list(hajek_probabilities_50_50_drta_base, hajek_probabilities_75_50_drta_base,
                                                                                               hajek_probabilities_95_50_drta_base))
stopCluster(cl)

cl <- makeSOCKcluster(n_clusters, outfile=paste0(c(data_directory, 'un_u250_logs.txt'), sep='', collapse=''))
registerDoSNOW(cl)
graph_randomized_un_u250 <- foreach(i = 1:n_simulations, 
                                    .combine = rbind,
                                    .multicombine= TRUE,
                                    .packages = c('dplyr', 'Matrix', 'blockTools', 'lfe', 'igraph'),                                    
                                    .maxcombine = 2) %dopar% simulate_graph_randomization(airbnb_listings, 
                                                                                          .5, 'unobserved', .75, 
                                                                                          sim_info,
                                                                                          .25*max_n_searchers, i, 
                                                                                          'normal', 
                                                                                          extreme_value_distribution,
                                                                                          clusters_distance_room_type_accom,
                                                                                          graph_distance_room_type_accom,
                                                                                          blocks_base,
                                                                                          c('prob_treat_50_thresh_base', 'prob_control_50_thresh_base',
                                                                                            'prob_treat_75_thresh_base', 'prob_control_75_thresh_base',
                                                                                            'prob_treat_95_thresh_base', 'prob_control_95_thresh_base'),
                                                                                          list(hajek_probabilities_50_50_drta_base, hajek_probabilities_75_50_drta_base,
                                                                                               hajek_probabilities_95_50_drta_base))
stopCluster(cl)

cl <- makeSOCKcluster(n_clusters, outfile=paste0(c(data_directory, 'un_u500_logs.txt'), sep='', collapse=''))
registerDoSNOW(cl)
graph_randomized_un_u500 <- foreach(i = 1:n_simulations, 
                                    .combine = rbind,
                                    .multicombine= TRUE,
                                    .packages = c('dplyr', 'Matrix', 'blockTools', 'lfe', 'igraph'),                                    
                                    .maxcombine = 2) %dopar% simulate_graph_randomization(airbnb_listings, 
                                                                                          .5, 'unobserved', .75, 
                                                                                          sim_info,
                                                                                          .5*max_n_searchers, i, 
                                                                                          'normal', 
                                                                                          extreme_value_distribution,
                                                                                          clusters_distance_room_type_accom,
                                                                                          graph_distance_room_type_accom,
                                                                                          blocks_base,
                                                                                          c('prob_treat_50_thresh_base', 'prob_control_50_thresh_base',
                                                                                            'prob_treat_75_thresh_base', 'prob_control_75_thresh_base',
                                                                                            'prob_treat_95_thresh_base', 'prob_control_95_thresh_base'),
                                                                                          list(hajek_probabilities_50_50_drta_base, hajek_probabilities_75_50_drta_base,
                                                                                               hajek_probabilities_95_50_drta_base))
stopCluster(cl)

In [ ]:
save(graph_randomized_un_u500, graph_randomized_un_u250, 
     graph_randomized_pn_u500, graph_randomized_pn_u250, 
    file = paste0(
         c(data_directory, 'graph_randomized_results_vary_u.Rdata'),
         sep='', collapse=''
     ))

In [ ]:
rm(hajek_probabilities_50_50_drta_base, hajek_probabilities_75_50_drta_base, hajek_probabilities_95_50_drta_base)

In [67]:
load(paste0(
         c(data_directory, 'data_for_simulation_blocked_washington.Rdata'),
         sep='', collapse=''
     ))

In [68]:
set.seed(1584021)
sim_info_wash <- vector("list", n_simulations)
searcher_preferences_uniform_wash <- vector("list", n_simulations)
lat_search_bounds_wash <- vector("list", n_simulations)
lon_search_bounds_wash <- vector("list", n_simulations)
max_lat_wash = quantile(airbnb_listings_washington$latitude, .9975)
min_lat_wash = quantile(airbnb_listings_washington$latitude, .0025)
max_lon_wash = quantile(airbnb_listings_washington$longitude, .9975)
min_lon_wash = quantile(airbnb_listings_washington$longitude, .0025)
min_acc_wash = min(airbnb_listings_washington$accommodates)
max_acc_wash = 4

cl <- makeSOCKcluster(n_clusters)
registerDoSNOW(cl)
sim_info_wash <- foreach(i=1:n_simulations, .packages=c('EnvStats', 'Matrix', 'dplyr', 'mvtnorm'),
                    .multicombine= TRUE, .maxcombine = 2) %dopar% {
    sim_info_wash <- list(search_weights = invisible(prop.table(runif(9))),
                     lat_search_bounds = invisible(matrix(runif(2*max_n_searchers, min=min_lat_wash, 
                                                                max=max_lat_wash),
                                                          ncol = 2)),
                     lon_search_bounds = invisible(matrix(runif(2*max_n_searchers, min=min_lon_wash, 
                                                                max=max_lon_wash),
                                                          ncol = 2)),
                     min_search_acc = runif(max_n_searchers, min=min_acc_wash, max=max_acc_wash),
                     search_quality_cutoff = runif(max_n_searchers, min=-2, max=2),
                     unif_searcher_weights = apply(matrix(runif(9*max_n_searchers), nrow=max_n_searchers), 
                                                   MARGIN=1, FUN=prop.table),
                     income = rnorm(max_n_searchers),
                     user_preferences_normal = rmvnorm(mean=rep(0,9), n=max_n_searchers)
                     )
}
stopCluster(cl)

extreme_value_distribution_wash = as(matrix(revd(n=nrow(airbnb_listings_washington)*max_n_searchers),
                                                            nrow=max_n_searchers), 'sparseMatrix')

In [69]:
set.seed(158402)
airbnb_listings_washington$unobserved_component <- rnorm(nrow(airbnb_listings_washington))
airbnb_listings_washington$price_scaled = as.numeric(scale(airbnb_listings_washington$price, scale=TRUE))
airbnb_listings_washington$rn <- seq(1, nrow(airbnb_listings_washington), 1)

In [60]:
n_clusters = 26
set.seed(12305980)
#Establish baselines

cl <- makeSOCKcluster(n_clusters)
registerDoSNOW(cl)
treatment_reality_pn_1000_wash <- foreach(i = 1:n_simulations, 
                                    .combine = rbind,
                                    .multicombine= TRUE,
                                    .packages = c('dplyr', 'Matrix'),                                     
                                    .maxcombine = 2) %dopar% simulate_ind_randomization_wash(airbnb_listings_washington, 
                                                                                  1,
                                                                                  'price',
                                                                                  .75,
                                                                                  sim_info,
                                                                                  max_n_searchers,
                                                                                  i,
                                                                                  'normal',
                                                                                       extreme_value_distribution)
stopCluster(cl)

cl <- makeSOCKcluster(n_clusters)
registerDoSNOW(cl)
control_reality_pn_1000_wash <- foreach(i = 1:n_simulations, 
                                    .combine = rbind,
                                    .multicombine= TRUE,
                                    .packages = c('dplyr', 'Matrix'),                                   
                                    .maxcombine = 2) %dopar% simulate_ind_randomization_wash(airbnb_listings_washington, 
                                                                                  0,
                                                                                  'price',
                                                                                  .75,
                                                                                  sim_info,
                                                                                  max_n_searchers,
                                                                                  i,
                                                                                  'normal',
                                                                                       extreme_value_distribution)
stopCluster(cl)

cl <- makeSOCKcluster(n_clusters)
registerDoSNOW(cl)
treatment_reality_un_1000_wash <- foreach(i = 1:n_simulations, 
                                    .combine = rbind,
                                    .multicombine= TRUE,
                                    .packages = c('dplyr', 'Matrix'),                                     
                                    .maxcombine = 2) %dopar% simulate_ind_randomization_wash(airbnb_listings_washington, 
                                                                                  1,
                                                                                  'unobserved',
                                                                                  .75,
                                                                                  sim_info,
                                                                                  max_n_searchers,
                                                                                  i,
                                                                                  'normal',
                                                                                       extreme_value_distribution)
stopCluster(cl)

cl <- makeSOCKcluster(n_clusters)
registerDoSNOW(cl)
control_reality_un_1000_wash <- foreach(i = 1:n_simulations, 
                                    .combine = rbind,
                                    .multicombine= TRUE,
                                    .packages = c('dplyr', 'Matrix'),                                   
                                    .maxcombine = 2) %dopar% simulate_ind_randomization_wash(airbnb_listings_washington, 
                                                                                  0,
                                                                                  'unobserved',
                                                                                  .75,
                                                                                  sim_info,
                                                                                  max_n_searchers,
                                                                                  i,
                                                                                  'normal',
                                                                                       extreme_value_distribution)
stopCluster(cl)

In [62]:
save(treatment_reality_pn_1000_wash, control_reality_pn_1000_wash, 
     treatment_reality_un_1000_wash, control_reality_un_1000_wash, 
    file = paste0(
         c(data_directory, 'baseline_results_wash.Rdata'),
         sep='', collapse=''
     ))

In [63]:
set.seed(120498)
#Establish baselines

cl <- makeSOCKcluster(n_clusters)
registerDoSNOW(cl)
bernoulli_un_1000_wash <- foreach(i = 1:n_simulations, 
                                    .combine = rbind,
                                    .multicombine= TRUE,
                                    .packages = c('dplyr', 'Matrix'),                                     
                                    .maxcombine = 2) %dopar% simulate_ind_randomization_wash(airbnb_listings_washington, 
                                                                                  .5,
                                                                                  'unobserved',
                                                                                  .75,
                                                                                  sim_info_wash,
                                                                                  max_n_searchers,
                                                                                  i,
                                                                                  'normal',
                                                                                       extreme_value_distribution_wash)
stopCluster(cl)

cl <- makeSOCKcluster(n_clusters)
registerDoSNOW(cl)
bernoulli_pn_1000_wash <- foreach(i = 1:n_simulations, 
                                    .combine = rbind,
                                    .multicombine= TRUE,
                                    .packages = c('dplyr', 'Matrix'),                                     
                                    .maxcombine = 2) %dopar% simulate_ind_randomization_wash(airbnb_listings_washington, 
                                                                                  .5,
                                                                                  'price',
                                                                                  .75,
                                                                                  sim_info_wash,
                                                                                  max_n_searchers,
                                                                                  i,
                                                                                  'normal',
                                                                                       extreme_value_distribution_wash)
stopCluster(cl)

In [64]:
save(bernoulli_un_1000_wash, bernoulli_pn_1000_wash, 
    file = paste0(
         c(data_directory, 'bernoulli_results_wash.Rdata'),
         sep='', collapse=''
     ))

In [66]:
load(paste0(
         c(data_directory, 'hajek_probabilities_blocked_base_wash.Rdata'),
         sep='', collapse=''
     ))
n_clusters <- 18


In [70]:
set.seed(12041246)
cl <- makeSOCKcluster(n_clusters)
registerDoSNOW(cl)
graph_randomized_pn_wash <- foreach(i = 1:n_simulations, 
                                    .combine = rbind,
                                    .multicombine= TRUE,
                                    .packages = c('dplyr', 'Matrix', 'blockTools', 'lfe', 'igraph'),                                    
                                    .maxcombine = 2) %dopar% simulate_graph_randomization_wash(airbnb_listings_washington, 
                                                                                          .5, 'price', .75, 
                                                                                          sim_info_wash,
                                                                                          max_n_searchers, i, 
                                                                                          'normal', 
                                                                                          extreme_value_distribution_wash,
                                                                                          clusters_distance_room_type_accom_washington,
                                                                                          graph_distance_room_type_accom_washington,
                                                                                          blocks_base_washington,
                                                                                          c('prob_treat_50_thresh_base', 'prob_control_50_thresh_base',
                                                                                            'prob_treat_75_thresh_base', 'prob_control_75_thresh_base',
                                                                                            'prob_treat_95_thresh_base', 'prob_control_95_thresh_base'),
                                                                                          list(hajek_probabilities_50_50_drta_base_wash, hajek_probabilities_75_50_drta_base_wash,
                                                                                               hajek_probabilities_95_50_drta_base_wash))
stopCluster(cl)

In [72]:
set.seed(12049015)
cl <- makeSOCKcluster(n_clusters)
registerDoSNOW(cl)
graph_randomized_un_wash <- foreach(i = 1:n_simulations, 
                                    .combine = rbind,
                                    .multicombine= TRUE,
                                    .packages = c('dplyr', 'Matrix', 'blockTools', 'lfe', 'igraph'),                                    
                                    .maxcombine = 2) %dopar% simulate_graph_randomization_wash(airbnb_listings_washington, 
                                                                                          .5, 'unobserved', .75, 
                                                                                          sim_info_wash,
                                                                                          max_n_searchers, i, 
                                                                                          'normal', 
                                                                                          extreme_value_distribution_wash,
                                                                                          clusters_distance_room_type_accom_washington,
                                                                                          graph_distance_room_type_accom_washington,
                                                                                          blocks_base_washington,
                                                                                          c('prob_treat_50_thresh_base', 'prob_control_50_thresh_base',
                                                                                            'prob_treat_75_thresh_base', 'prob_control_75_thresh_base',
                                                                                            'prob_treat_95_thresh_base', 'prob_control_95_thresh_base'),
                                                                                          list(hajek_probabilities_50_50_drta_base_wash, hajek_probabilities_75_50_drta_base_wash,
                                                                                               hajek_probabilities_95_50_drta_base_wash))
stopCluster(cl)

In [73]:
save(graph_randomized_pn_wash, graph_randomized_un_wash, 
    file = paste0(
         c(data_directory, 'graph_randomized_results_wash.Rdata'),
         sep='', collapse=''
     ))